## Project D1: Results of the Occupation Index Autocoder

Kelly Wilkin
[kelly.r.wilkin@census.gov](mailto:kelly.r.wilkin@census.gov)
June 9, 2021

### Introduction
The American Community Survey (ACS) survey instrument asks employed sample members to report their job title and typical job duties in two open-ended text fields. Respondents are free to enter any text string in these fields and clerical coders at the Census Bureau's National Processing Center (NPC) convert that information into numeric Census occupation codes. The Industry and Occupation Statistics Branch (IOSB) has identified this process as a useful application of machine learning (ML), where models can be trained to predict a given respondent's occupation code from these write-in fields without clerical coder intervention. Successful prediction of occupation codes would speed processing time and save valuable NPC resources.

Exploratory analysis of ACS occupation write-in fields shows many write-in values replete with spelling errors, incomplete words, and inconsistent use of punctuation, all of which complicate data preparation for predictive machine learning models. However, many write-in values for job titles appear to be written verbatim to known occupation code assignments. For example, Census maintains the [Census Occupation Index](https://www2.census.gov/programs-surveys/demo/guidance/industry-occupation/Alphabetical-Index-of-Occupations-December-2019_Final.xlsx) ("index" hereafter), a collection of about 33,000 occupation titles and their associated Census occupation codes. In addition, the index makes explicit other restrictions (or rules) required for a given job title to be matched to the defined Census code, such as education level, industry, and class of worker. To the extent that index job titles match ACS occupation write-in fields, the index can be used as a deterministic autocoder to provide highly reliable code assignment while simultaneously controlling for any explicit restrictions imposed on a given job title; that is, autocodes can be assigned from a basic look-up routine using an existing source of accurate labeled data without reliying on stochastic predictions from a model. A potential autocoder workflow could include first matching write-in fields with the index to pick off easy matches, then leave the remaining unmatched write-ins to be predicted using ML modeling. If stochastic features of any ML model will predict some write-ins with error and those write-ins could have been matched by the index, incorporating the index as an intermediate step in prediction will increase overall autocoding accuracy while reducing prediction time and building into the process consistency checks with education, class of worker, and industry variables.

The utility of the index as part of the autocoder pipeline depends on the rate at which occupation write-ins match index job titles. Operationalizing the index for exact matches requires cleaning of the index itsef and constructing software-specific scripts to account for any explicit restrictions (e.g., Python or SAS syntax). In its current state, the index may describe education restrictions in either the job title itself or as part of industry restrictions. For the most part, entries in the index follow consistent patterns that can be exploited to extract information both about restrictions but to also clean the job titles. Such efforts may not be worthwhile if write-ins match with low frequency. 

This `jupyter` notebook presents results from a first attempt at creating a deterministic matching tool from the occupation index. Using the internal ACS 2018 and 2019 one-year files, I evaluate the incidence of occupation code assignment using a simple algorithm that requires a write-in value to match verbatim job titles in the index while also controlling for industry, class of worker, and education restrictions outlined therein. My strategy to use convert any restrictions into a logical condition that resolves to either `True` or `False` when taken with the inputs from a given occupation write-in and the values of industry, occupation, and class of worker for the same record. Any write-in value that resolves to `True` is a candidate for an autocode value; cases with only one matching industry is assigned that value.

Have data on whether existing autocoder (logistic regression) is able to make a prediction. If it does not but the deterministic approach does, then that provides an estimate of the incidence of write-ins that were sent to clerical coding that did not have to. This is one measure of cost savings.

I find these results.

This shows that we can achieve almost half the performance of the current autocoder using data from the index without specifying any stochastic model. This amounted to X number of cases that ended up going through clerical coding. Further, 

These results show that incorporating an exact matching routine as a first step in the ACS occupation autocode pipeline can improve upon the performance of the existing autocode, produce highly reliable autocodes, and reduce the set of records that must be fed into a prediction model.

The rest of this notebook is organized as follows. In the next section I briefly describe the occupation index and how patterns are leveraged within it to construct `python`-syntactic conditions to either control for explicit restrictions or rule them out. I discuss some simple patterns built into the index design that simplify identification of restrictions, as well as additional index design features could make it more useful. Then I present a simple algorithm for exact matching. Then I present results. Then I summarise.

Throughout the document I will work directly with the ACS sample. This will allow me to present the exact matching workflow in a production environment (using `python`) while adding discussion where possible. It is important to remember that is simply a first pass or "proof of concept." The quality of matches will be only as good as the inde

### Data
I test the exact match autocoder on the internal ACS 2018 and 2019 one-year files. This section discusses how each file is loaded into the Python environment and prepared for the exact match autocoder. I begin by loading the data and dropping all records with missing data in any of our required fields; these include missing write-ins for either occupation write-in field, the target occupation code, and inidicators for industry, educational attainment, and class of worker. Records with missing occupation codes are likely to be out of universe for occupation code variables, and no matching is possible for cases where no write-in was given. Because the exact match autocoder depends on equality between write-in fields to job titles in the occupation index, no additional pre-processing is required. We end by printing a small subsample of write-ins to illustrate the data.

ACS data are stored as SAS data files, which can be loaded directly into Python using the `pandas` library. It is important to read in the text data correctly so that differences in the encoding of the text fields are interpreted correctly by Python. This is controlled by specifying the encoding parameter in the `read_sas()` function as `encoding = ISO-8859-1` (there are other encoding types, such as UTF-8, but I have found that ISO-8859-1 works best with SAS data sets). Alternatives are to use other methods that convert SAS data sets into other formats known to work across systems (e.g., CSV), but that is an extra step and stresses memory limits by duplicating data on disk.

The cell below shows the process for loading each data file, dropping records with missing values for certain variables, and combining each file into a single data set. The file paths to each file are based on the survey year. I define a simple dictionary `acsdata` that links a survey year to its file path within the IRE. We then loop over each year and import the data file; each data file is stored in a dataframe with the naming convention `dfYY`, where `YY` is the two-digit year indicator (e.g., `YY` = 18 for 2018). The list object `keep_features` defines the set of variables for which all records should have non-missing data. We then loop over each file to delete missing records and add a year indicator linking a given record to its source file. Along the way, we tally the number of records for each file before and after deleting cases with missing values. Finally, we combine the files into one large file (`acs`) and drop the intermediate individual files.

In [1]:
# import modules
import os
import json
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# define location of source acs
datroot = '/data/demographic/acswritein'

# get list of acs data files (and paths), store in dictionary
acsyears = [ 2018 , 2019 ]

acsdata = dict()

for i in acsyears:
    path = os.path.join( datroot , str( i ) )
    datfile = [ x for x in os.listdir( path ) if 'sas7bdat' in x ][ 0 ]
    acsdata[ i ] = os.path.join( path , datfile )


# load acs data
for i in acsyears:
    globals()[ 'df{}'.format( i - 2000 ) ] = pd.read_sas( acsdata[ i ] , 
                                                          format = 'sas7bdat' ,
                                                          encoding = 'iso-8859-1' )

# drop records with missing features
keep_features = [ 'OCC' , 'ocw1' , 'ocw2' , 'IND' , 'inw2' , 'inw3' ,
                  'SCHL' , 'COW' ]

for i in acsyears:
    dfname = 'df{}'.format( i - 2000 )
    
    print( 'shape before dropping missing OCC and write-ins for {}:'.format( dfname ) , 
           globals()[dfname].shape )
    
    globals()[dfname].dropna( subset = keep_features ,
                              axis = 0 ,
                              inplace = True )
    
    globals()[dfname]['year'] = i
    
    globals()[dfname].reset_index(drop=True, inplace=True)
    
    print( 'shape after dropping missing records for {}:'.format( dfname ) , 
           globals()[dfname].shape )


# stack the years into a single acs file
acs = pd.concat( [ df18 , df19 ] , axis = 0 )

# delete individual annuals file from the global space to save memory
del df18 , df19

shape before dropping missing OCC and write-ins for df18: (4902428, 75)
shape after dropping missing records for df18: (2322035, 76)
shape before dropping missing OCC and write-ins for df19: (4718200, 70)
shape after dropping missing records for df19: (2211609, 71)


These results show the large number of occupation (and industry) write-ins needed for matching in a given survey year. There are just over 2.3 and 2.2 million write-in records for the 2018 and 2019 ACS files, respectively, for a combined 4.5 million total records to run through autocoding over this two-year period. These results highlight the production need for a reliable autocoder to reduce the workload being sent to the NPC for clerical coding.

The next cell shows presents a glimpse of what the write-in data along with several inputs that the exact match autocoder will use. The first 10 records of `acs` are printed below. The field `OCC` is the occupation code ultimately assigned by NPC clerical coders or the existing logistic regression autocoder. Assuming that these are the correct matches, `OCC` comprises "labels" that an autocoder will use to train a model to predict based on the write-in (and potentially other) fields, or "features." The write-in fields are `ocw1` and `ocw2`, where respondents record their job title and job tasks, respectively (as we will see later, both fields can be used for exact matching). The field `auto_uocc` stores the occupation code prediction from the current logistic regression production autocoder; values of `NaN` indicate "not a number" meaning no autocode assignment could be made for that record. The fields `IND`, `SCHL`, and `COW` store indicators for industry, educational attainment, and class of worker, respectively.

Evaluating any autocoder's accuracy amounts to comparing the label `OCC` to a predited autocode assignment. For our purposes, we will evaluate the exact match autocode to `OCC` as well as to `auto_uocc` in order to quantify its ability to correctly match codes when the production autocoder could not. But before we do that, we need to describe the occupation index.

In [3]:
# print the first 10 records of write-in and select inputs from the acs file
acs[['OCC','auto_uocc','ocw1','ocw2','IND','SCHL','COW']].head(10)

,OCC,auto_uocc,ocw1,ocw2,IND,SCHL,COW
0,4220,NaN,CONDO CLEANING,CLEANING CONDOS,7072,17,1
1,9620,NaN,LABORER,WAREHOUSE,7580,15,1
2,4220,NaN,MAINTENANCE,STOCKING/CLEANING,5381,16,1
3,9130,9130,DRIVING A TRUCK,DRIVING THE TRUCK WHILE STRIPING THE ROAD,0770,19,1
4,3422,NaN,BEHAVIORAL AIDE,TEACHING MENTAL PATIENTS BASIC SKILLS,8090,16,1
5,7260,NaN,TIRE/BATTERY SPECIALIST,CHANGING TIRES & BATTERIES,5391,16,1
6,4621,4621,PERSONAL TRAINING MANAGER,CLEARICAL WORK SHOWING MEMBERS ROUTINES AND EX...,8590,18,1
7,4110,NaN,SERVING FOOD AND MAKING COCKTAILS,SERVING FOOD AND MAKING COCKTAILS,8680,19,1
8,7340,NaN,MAINT,REPAIRS,9370,16,3
9,0440,NaN,MANAGER,MANAGING,8590,21,3


In [2]:
# load merged file from disk to avoid repeating the above
acs = pd.read_csv( 'acs_merged.csv' )

# print the first 10 records of write-ins and select inputs from the acs file
acs[['OCC','auto_uocc','ocw1','ocw2','IND','SCHL','COW']].head(10)

/apps/anaconda/envs/datascience/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (11,64,67,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,OCC,auto_uocc,ocw1,ocw2,IND,SCHL,COW
0,4220,NaN,CONDO CLEANING,CLEANING CONDOS,7072,17,1
1,9620,NaN,LABORER,WAREHOUSE,7580,15,1
2,4220,NaN,MAINTENANCE,STOCKING/CLEANING,5381,16,1
3,9130,9130.0,DRIVING A TRUCK,DRIVING THE TRUCK WHILE STRIPING THE ROAD,770,19,1
4,3422,NaN,BEHAVIORAL AIDE,TEACHING MENTAL PATIENTS BASIC SKILLS,8090,16,1
5,7260,NaN,TIRE/BATTERY SPECIALIST,CHANGING TIRES & BATTERIES,5391,16,1
6,4621,4621.0,PERSONAL TRAINING MANAGER,CLEARICAL WORK SHOWING MEMBERS ROUTINES AND EX...,8590,18,1
7,4110,NaN,SERVING FOOD AND MAKING COCKTAILS,SERVING FOOD AND MAKING COCKTAILS,8680,19,1
8,7340,NaN,MAINT,REPAIRS,9370,16,3
9,440,NaN,MANAGER,MANAGING,8590,21,3


In [3]:
acs.shape

(4533644, 77)

In [4]:
acs[['IND','OCC','auto_uocc']].dtypes

IND            int64
OCC            int64
auto_uocc    float64
dtype: object

### Occupation Index
The index comprises a set of job titles and their associated Census occupation codes, as well as select combinations of class of worker, education level, and industry that may be required for internal consistency. This section breifly describes the index and how pattern recognition tools (e.g., regular expressions) can be used to extract informative content from it. We then present results from a first pass at converting the index to Python syntactic code to operationalize the exact match autocoder in a later step.

The index is stored as an Excel file and posted on the Census Bureau's Web site. The `pandas` library can load it directly into Python. The following cell illustrates this process. Note that column names are changed to make them shorter and remove spaces, and records with write-ins but no assigned Census occupation code (denoted by field `cenocc`) are dropped. In addition, Census codes are represented as four-digit character strings. The default `pandas` settings automatically convert these to numeric (float) values, which removes leading zeroes. While perhaps unnecessary, I define a function that converts occupation codes back to strings and adds leading zeroes. Finally, the first 20 records of the index are printed.

In [4]:
# occupation index file path
occ_index_excel = 'Alphabetical-Index-of-Occupations-December-2019_Final.xlsx'

# load occupation index excel file as pandas data frame
occdf = pd.read_excel( open( occ_index_excel , 'rb' ) , 
                       index_col = None , 
                       skiprows = 6 )

# rename columns to be easier to work with
occdf.columns = [ 'job_title' , 'ind2017restr' , 'cenocc' , 'soc2018' ]

# count the number of records with missing 'cenocc'
print( 'number entries with missing cenocc:' , occdf['cenocc'].isna().sum() )

# count total records in the index after dropping missing cenocc
print( 'number of occupation titles in the index with cenocc:' , len( occdf ) )

# drop records with missing 'cenocc'
occdf.dropna(axis=0, subset=['cenocc'], inplace=True)
occdf.reset_index( drop = True , inplace = True )

# convert 'cenocc' to character and add leading zeroes
# (a) define function for adding leading zeroes to a 4-character string
def leadingzeroes( occ_code ):
    if len( occ_code ) == 4:
        return( occ_code )
    else:
        return( '{0}{1}'.format( '0' * ( 4 - len( occ_code ) ) , occ_code ) )

# (b) add leading zeroes to cenocc and print result
occdf['cenocc'] = occdf['cenocc'].apply(lambda x: leadingzeroes(str(x).split('.')[0]))

occdf.head(20)

number entries with missing cenocc: 281
number of occupation titles in the index with cenocc: 32953


,job_title,ind2017restr,cenocc,soc2018
0,Admiral,NaN,0010,11-1011
1,Board chairman,NaN,0010,11-1011
2,Board member,NaN,0010,11-1011
3,Bureau chief,9370-9590,0010,11-1011
4,CEO (chief executive officer),NaN,0010,11-1011
5,CFO (chief financial officer),NaN,0010,11-1011
6,Chairman of the board,NaN,0010,11-1011
7,Chancellor,7870,0010,11-1011
8,Chief executive officer (CEO),NaN,0010,11-1011
9,Chief financial officer (CFO),NaN,0010,11-1011


The index comprises nearly 33,000 occupation titles and their associated occupation codes. Job titles have no occupation coding for 281 cases. However, the above exercise demonstrates that, should those codes be added at a later date, it's a simple matter of reloading the index to incorporate them in updated versions of this exact match autocoder.

The first 20 job titles printed above show the reltionship between job title (`job_title`), Census occupation code (`cenocc`), and any industry or class of worker restrictions (`ind2017restr`); the 2018 SOC code is also provided but not needed for our purposes since `cenocc` is already indicated. The strategy is to treat job titles as write-ins. If a write-in matches the job title exactly, then the value of `cenocc` is assigned to that record, otherwise that record will be set aside to go through a predicted model based on more advanced ML techniques.

The printed subsample demonstrates some of the coding challenges and opportunities in using the index. First, occuation titles include text that is unlikely to appear as written. For example, the entry for "CEO" (at index = 4) is spelled out as "chief executive officer" in parentheses, but the entry at index = 8 is "chief executive officer" spelled out with "CEO" in parentheses. While this appears to be a case of duplicate entries (note also that all other fields for these entries are identical), these entries actually represent two distinct ways that respondents my write their job title. Further, it is unlikely that they would include the parentheses and additional text. I make the assumption that matching rates would be improved by deleting additional text within parentheses on job titles, and thus include this a pre-processing step.

Second, industry restrictions (column `ind2017restr`) comprise both industry and class of worker restrictions and do so in different ways. For example, the entry for "Chancellor" (index = 7) lists only a single industry restriction (i.e., `IND` = 7870), while that for "Bureau chief" lists a range of values (`IND` = 9370-9590). In addition, the industry restriction for "College president" (index = 18) is preceded by a "#" symbol, giving it special priority for records with this job title. It is possible to construct a function that takes these inputs and simultaneously controls for all of them in order to find---hopefully---just one matching occupation code for assignment. 

Not pictured in the above index examples are education-level restrictions that primarily pop-up in the job titles but may also be referenced in industry restrictions. Education restrictions are broadly applied to a given write-in and do not appear to be associated with any specific set of industry combinations. This simplifies the coding process since we can rule out any host of industry or class-of-worker restrictions if the education restriction fails. In the next cell of this notebook, I use a `regex` to flag records with an explicit education restriction to demonstrate index entries that contain them. Regular expressions can be quite complicated and *ad hoc* and the one chosen below (defined in the variable `repat`) is based on a short survey of index entries.

In [5]:
acs['IND'] = acs['IND'].apply(lambda x: leadingzeroes(str(int(x))))
acs['OCC'] = acs['OCC'].apply(lambda x: leadingzeroes(str(int(x))))

In [6]:
# define regular expression for having an education restriction
repat = '(degree)|((?<=\s)diploma)'

# apply the regex to 'job_title' and flag index records matching the regex
occdf['hased'] = np.where(occdf['job_title'].apply(lambda x: len(re.findall(repat,str(x),flags=re.I)))>0,
                          1,
                          np.where(occdf['ind2017restr'].apply(lambda x: len(re.findall(repat,str(x),flags=re.I)))>0,
                                   1,
                                   0))

# count the index entries with education restriction
print('number of index records with explicit education restriction:' ,
      len(occdf[occdf['hased']==1]))

number of index records with explicit education restriction: 100


The above `regex` searches within the `job_title` and `ind2017restr` fields for the terms "degree" or "diploma" and returns a match if either of those terms appear in the string. The pattern identified 100 index records with explicit education restrictions. I print below the first 20

This small sample shows that, while most education restrictions are listed in the job title, they may also appear in the industry restriction as depicted by index entry 1682. Further, note how education values may take on a range, such as "associate degree or higher" or "less than a professional degree." I will be able to exploit these patterns to link these ranges to specific numeric values appearing in the ACS variable `SCHL`. Numeric values for education ranges are provided in the ACS 1-year technical documentation. The mapping of specific education-level references to their numeric values is handled by a user-defined function presented later in this notebook.

The final challenge to using job titles as autocoder entries with education restrictions is removing those restrictions from the job title. Recall that the exact match autocoder seeks to link write-ins with verbatim matches from the index. It is unlikely that a "machine designer" will also record their education level in the same string (see index entries 2652 and 2775 below). A `regex` can be used to purge educational content from the job title.

In [84]:
occdf[occdf.hased==1].head(20)

,job_title,ind2017restr,cenocc,soc2018,hased
246,Bank cashier associate degree or higher,#6870,0120,11-3031,1
252,Cashier bank associate degree or higher,#6870,0120,11-3031,1
253,Cashier\ ns associate degree or higher,6870,0120,11-3031,1
1405,Medical examiner less than professional degree,(7970),0565,13-1041,1
1681,Account or accountant bookkeeper associate deg...,NaN,0800,13-2011,1
1682,Accountant,bachelor's degree or higher,0800,13-2011,1
1687,Accounting work associate degree or higher,NaN,0800,13-2011,1
2250,Statistical work associate degree or higher,NaN,1240,15-20XX,1
2289,Land surveyor education bachelors degree or hi...,NaN,1310,17-1020,1
2310,Surveyor surveying land bachelors degree or hi...,NaN,1310,17-1020,1


I identify only two index entries where educational information is stored in `ind2017restr`. These are presented below. Note that no other information is contained in these entries, thus additional cleaning or purging of education levels from these restriction is unnecessary.

In [85]:
occdf[(occdf.hased==1) & (occdf.ind2017restr.apply(lambda x: len(re.findall(repat,str(x),flags=re.I)))>0)]

,job_title,ind2017restr,cenocc,soc2018,hased
1682,Accountant,bachelor's degree or higher,0800,13-2011,1
11126,Accountant,less than bachelor's degree,5120,43-3031,1


### Processing the Index
Now that we have a clearer understanding of the information contained in the index, it's time to operationalize it for use as an exact match autocoder. In this section, I develop a strategy for using the index as an autocoder. It is simple. First, clean the index to remove non-title content from job titles. Second, convert any restrictions---education, class of worker, industry---into code that the Python (or other) interpreter can use to check whether any conditions are present, evaluate them if so, and choose the occupation code(s) that apply to a write-in. I construct a set of user-defined functions for each of these steps. Finally, apply the exact match autocode to the write-in data.

Before getting into the details, it is useful to present an example of a fully processed index entry ready to be used as an autocoder. The general algorithm of the autocoder is to create a set of booleans where, if all resolve to `True`, that index entry is a suitable match for a write-in. Since the restrictions may be based on education, class of worker, and industry the booleans must be able to take values of each from ACS write-in fields and evaluate them against the index.


#### Cleaning Job Titles
Occupation index job titles often contain information that, while useful for flagging certain restrictions, is unlikely to be included as part of a write-in. Examples presented above show that this can include acronyms (e.g., "CEO" in parentheses following a complete spelling of "chief executive officer" and vice versa), exclusion restrictions written as text, and references to explicit education restrictions. In addition, character strings, such as those indicating "not specified" (i.e., "\ ns"), may appear and would not be expected to in a write-in field. This section presents a simple method for cleaning job titles of extraneous content.

Cleaning exploits two patterns to purge extraneous content from job titles. First, I identify the of of parentheses and simply "delete" the parentheses and all content between them by replacing those string elements with null space. Second, I use the fact that most restrictions (e.g., education) tend to be the last part of the job title. Therefore, any text that precedes explicit education references is assumed to comprise the actual job title. Using the `re.split()` function, we can flag the pattern where education (or exclusion) references occur, split the string on that pattern, then return all of the text preceding the split. I use a simliar approach to split on the "\" that often precedes references to "not specified," as well as "any."

Cleaning is handled by the `cleanTitle()` function displayed below. It takes a single input `input_string` and returns the string purged of text meeting patterns specified within the function. If any pattern is not found, the `input_string` is returned as-is.

In [7]:
def cleanTitle(input_string):
    # split on '\'
    text = re.split( '\\\\' , input_string )[ 0 ]
    
    # remove text in parentheses
    repat = '(\([a-z\s\/\-]+(?=\))\)$)|(\\\\\s?n\s?s)|((?<=\s)[a-z]+\sdegree[\sa-z\']+$)'
    
    text = re.sub( repat , '' , text , flags = re.I ).strip()
    
    # remove education level if still present
    # NOTE: the term 'education' shows up in 'land surveyor' entries. 'education' shows up in many titles and
    #       we must be selective about deleting any instance of 'education'
    repat = '\s((?<=surveyor\s)education|less than|with|without|\(?exc\s|other\sspecified|high\sschool|associates|professional|masters?|bachelors?)'
    text = re.split( repat , text , flags = re.I )[ 0 ]
    
    # split on 'any' if in job title
    repat = '\sany(?=\s)'
    text = re.split( repat , text , flags = re.I )[ 0 ]
    
    return( text  )

Next, I apply `cleanTitle` to the occupation index field `job_title` and store the result in a new field `title` using a lambda function. I also present the first 20 records of the index showing original job titles and their cleaned versions.

In [8]:
occdf['title'] = occdf['job_title'].apply(lambda x: cleanTitle(x))

occdf[['job_title','title','cenocc']].head(20)

,job_title,title,cenocc
0,Admiral,Admiral,0010
1,Board chairman,Board chairman,0010
2,Board member,Board member,0010
3,Bureau chief,Bureau chief,0010
4,CEO (chief executive officer),CEO,0010
5,CFO (chief financial officer),CFO,0010
6,Chairman of the board,Chairman of the board,0010
7,Chancellor,Chancellor,0010
8,Chief executive officer (CEO),Chief executive officer,0010
9,Chief financial officer (CFO),Chief financial officer,0010


The above shows how the `cleanText()` function sucessfully removed text within parentheses. Next, let's look at entries comprising education restrictions to see that job titles were properly extracted. Below I show original and cleaned job titles the first 20 records for cases with an education restriction in the job title.

In [9]:
occdf[occdf.hased==1][['job_title','title','cenocc']].head(20)

,job_title,title,cenocc
246,Bank cashier associate degree or higher,Bank cashier,0120
252,Cashier bank associate degree or higher,Cashier bank,0120
253,Cashier\ ns associate degree or higher,Cashier,0120
1405,Medical examiner less than professional degree,Medical examiner,0565
1681,Account or accountant bookkeeper associate deg...,Account or accountant bookkeeper,0800
1682,Accountant,Accountant,0800
1687,Accounting work associate degree or higher,Accounting work,0800
2250,Statistical work associate degree or higher,Statistical work,1240
2289,Land surveyor education bachelors degree or hi...,Land surveyor,1310
2310,Surveyor surveying land bachelors degree or hi...,Surveyor surveying land,1310


Take note of index entries 2652 and 2775 for "Machine designer." Note how the former, which has Census occupation code 1460, is reserved for machine designers with an associate degree or higher, while the latter (occupation code 1545) is assigned to cases with less than an associate degree. Clearly, ACS write-ins matching "machine designer" will match twice. Without additional information on educational attainment the appropriate code cannot be assigned. However, these cleaned entries are more likely to match a write-in than the original `job_title`. Next we present a method for storing education restrictions so that the write-ins can choose among candidate occupation codes with such requirements.

#### Capturing Education Restrictions
I showed previously how education restrictions appear in job titles as a text string description of one or more education levels. ACS data store education levels as a numeric code (e.g., `SCHL`). This section presents a method for converting education restriction text to an explicit Python condition that can take the value of `SCHL` on the same record as a given write-in and evaluate whether the education restriction is met. I deifne a function that takes any input string from the index, identifies whether it has an education component and, if so, converts education references into numeric values. In addition, it accounts for education ranges, such as "less than" a certain education level or "higher." Further, it can handle negatives, such as "without a professional degree." In cases where an education level is found, the function returns a character string that can be parsed into Python code, otherwise it returns a value of `None`.

As before, I use a `regex` to identify education patterns and a the ACS 1-year PUMS technical documentation to associate education levels with numeric values. I use different `regex` patterns to identify references to ranges of education levels. These patterns were determined to be exhaustive of unique cases involving an education restriction based on the criterion of a text string containing the string "degree" or "diploma." I then construct an output string with a free field that can later be used to substitute explicit values of `SCHL` for a given record.

The function `edrestr` is presented below.

In [10]:
# FUNCTION TO GET PYTHON SYNTAX FOR EDUCATION RESTRICTION
def edrestr( input_string ):
    # check if input_string has an education requirement
    def hased( edstring ):
        if pd.isnull( edstring ):
            return( 0 )
        else:
            if len( re.findall( '(degree)|((?<=\s)diploma)' , 
                                edstring ,
                                flags = re.I ) ) > 0:
                return( 1 )
            else:
                return( 0 )
    
    # input_string has an education restriction
    if hased( input_string ):
        # define dictionary of 'SCHL' values
        degree_vals = { 'high school' : [ 16 , 17 ] ,
                        'some college' : [ 18 , 19 ] ,
                        'associate' : 20 ,
                        'bachelor' : 21 ,
                        'master' : 22 ,
                        'professional' : 23 ,
                        'doctorate' : 24 }
        
        # regex pattern to identify presence of education restriction
        repat = '(high school|some college|bachelor|professional|associate|master)'
        
        # run regex on input sting
        hasdeg = re.findall( repat , input_string , flags = re.I )
            
        if len( hasdeg ) == 1:
            degree = hasdeg[ 0 ].lower()
            degval = degree_vals[ degree ]
            degtype = isinstance( degval , ( list ) )
            
            ltlvl = re.findall('(less\sthan|or\sless)', input_string, flags=re.I )
            gtlvl = re.findall('or\s(higher|more|greater)', input_string, flags=re.I )
            wolvl = re.findall('without(?=\s?{0})'.format(degree), input_string, flags=re.I )
            wtlvl = re.findall('with(?=\s?{0})'.format(degree), input_string, flags=re.I )
            
            if len( gtlvl ) > 0:
                if degtype:
                    return( '{{0}}>={}'.format( max( degval ) ) )
                else:
                    return( '{{0}}>={}'.format( degval ) )
            elif len( ltlvl ) > 0:
                if degtype:
                    return( '{{0}}<{}'.format( min( degval ) ) )
                else:
                    return( '{{0}}<{}'.format( degval ) )
            elif len( wolvl ) > 0:
                if degtype:
                    return( '{{0}} not in {}'.format( degval ) )
                else:
                    return( '{{0}}!={}'.format( degval ) )
            else:
                if degtype:
                    return( '{{0}} in {}'.format( degval ) )
                else:
                    return( '{{0}}=={}'.format( degval ) )
        elif len( hasdeg ) > 1:
            patlst = ['{{0}} in {}'.format(degree_vals[x]) if isinstance(degree_vals[x], (list)) 
                      else '{{0}}=={}'.format(degree_vals[x]) for x in hasdeg]
            return( ' or '.join( patlst ) )
    else:
        pass

Next, I demonstrate `edrestr` on the set of fields with explicit education restrictions. Recall that education references appear in `job_title` or `ind2017restr`. I use the variable `hased` that we defined earlier and then apply the `np.where` function to conditionally check across each variable. I then present the function result along with the original and cleaned job titles for the first 20 index entries that have an education component.

In [11]:
# conditionally execute function on records with an education restriction
occdf['edrestr'] = np.where(occdf.hased==1,
                            np.where(occdf.job_title.apply(lambda x:len(re.findall(repat,str(x),flags=re.I)))>0,
                                     occdf.job_title.apply(lambda x: edrestr(str(x))),
                                     occdf.ind2017restr.apply(lambda x: edrestr(str(x)))),
                            None)

# print first 20 records
occdf[occdf.hased==1][['job_title','ind2017restr','title','edrestr','cenocc']].head(20)

,job_title,ind2017restr,title,edrestr,cenocc
246,Bank cashier associate degree or higher,#6870,Bank cashier,{0}>=20,0120
252,Cashier bank associate degree or higher,#6870,Cashier bank,{0}>=20,0120
253,Cashier\ ns associate degree or higher,6870,Cashier,{0}>=20,0120
1405,Medical examiner less than professional degree,(7970),Medical examiner,{0}<23,0565
1681,Account or accountant bookkeeper associate deg...,NaN,Account or accountant bookkeeper,{0}>=20,0800
1682,Accountant,bachelor's degree or higher,Accountant,{0}>=21,0800
1687,Accounting work associate degree or higher,NaN,Accounting work,{0}>=20,0800
2250,Statistical work associate degree or higher,NaN,Statistical work,{0}>=20,1240
2289,Land surveyor education bachelors degree or hi...,NaN,Land surveyor,{0}>=21,1310
2310,Surveyor surveying land bachelors degree or hi...,NaN,Surveyor surveying land,{0}>=21,1310


Column `edrestr` presents the results of the `edrestr()` function for the first 20 index entries with an education restriction. The result is a character string relating an open field (denoted by `{0}`) to a value of `SCHL` in the ACS technical documentation corresponding to a stated education level. The open field is designed to take any value; the notation `{0}` is interpreted by Python's `format()` function as a portion of the string where some value can be substituted (the '0" is a positional argument that is not required in Python 3, but keeping it makes the code backward compatible with Python 2.7). 

My strategy is substitute the value of `SCHL` from a given ACS record and evaluate the condition as either `True` or `False`, indicating whether the education restriction is satisfied. Consider again the example for "Machine designer," which has entries on index 2652 and 2775. If a respondent enters their occupation (e.g., in the ACS field `ocw1`) as "Machine designer," it will match both entries referenced above. The choice of which Census occupation code to assign---1460 or 1545---depends on the respondent's reported education level. If such information is available in `SCHL`, then we can substitute it into the conditions of `edrestr` and select the occupation code for which the education restriction resolves to `True`.

Below I demonstrate this with a hypothetical respondent who enters on the ACS questionnaire "Machine designer" in `ocw1` and also reports having some college but less than one year (i.e., `SCHL` = 18). Based on the two options for "Machine designer" this entry should be assigned occupation code 1545 because this fictional respondent does not have an associate degree. Let's see how to select this using Python. 

In [90]:
# subset "machine designer" and evaluate 'edrestr' based on a hypothetical value of SCHL = 18
# set sample variable for 'schl' (this would be the ACS field in production)
schl = 18

# subset the records for "Machine designer" for example
test = occdf[occdf.title=='Machine designer'].copy()

# evaluate 'edrestr' at 'schl' = 18
test['eval_edrestr'] = test.edrestr.apply(lambda x: eval(x.format(schl)))

# print the result
print('show evaluation of edrestr at schl=18:')
test

show evaluation of edrestr at schl=18:


,job_title,ind2017restr,cenocc,soc2018,hased,title,edrestr,eval_edrestr
2652,Machine designer associate degree or higher,NaN,1460,17-2141,1,Machine designer,{0}>=20,False
2775,Machine designer less than associate degree,NaN,1545,17-3013,1,Machine designer,{0}<20,True


In [16]:
# choose code where eval_edtrestr is True for this sample case where schl = 18
test[test.eval_edrestr == True][['title','eval_edrestr','cenocc']]

,title,eval_edrestr,cenocc
2775,Machine designer,True,1545


In [21]:
# clean up temp variables
del schl, test

#### Class of Worker Restrictions
This section presents code for identifying class-of-worker restrictions in the index. I follow the same strategy used in the previous section, but instead look for patterns in `ind2017restr` and define a user-defined function to link specific restriction text to `COW` codes in the ACS and then return a text string that, when substituted with a value for `COW` associated with a write-in, can be parsed by Python and evaluated to `True` or `False`.

Below presents the `regex` pattern used to identify class-of-worker references in the occupation index. To simplify the exposition, I present examples on the set of index entries with a class-of-worker restriction.

In [14]:
# cow regex
cowpat = '(OWN|(?<![lsf])GOV|PR|OBNI|OBI|WP|LGOV|SGOV|FGOV)(?![a-z])'

# create a subset of records with a cow restriction
cowdf = occdf[occdf.ind2017restr.apply(lambda x: len(re.findall(cowpat, str(x), flags=re.I)))>0].copy()

# display 20 random records with a cow restriction
cowdf[['title','cenocc','ind2017restr']].sample(n=20).head(20)

,title,cenocc,ind2017restr
697,Attendant,0310,"OWN 8680, 8690"
565,Shellfish grower,0205,"GOV, OWN, PR 0180"
16,City superintendent,0010,LGOV 9370
563,Share worker,0205,OBNI 0170
466,Farm rancher,0205,"OWN 0170, 0180, 0290"
1731,City assessor,0810,#LGOV 9380
479,Farmer,0205,"OWN 0170, 0180, 0290"
13965,Quarryman,6200,OWN 0470
529,Manager production seed corn,0205,"GOV, OBI or PR 0170, 0290"
30677,Garageman,9365,"GOV or PR 6370, 8770, 9090"


The examples above reveal several challenges with flagging COW restrictions. First, multiple COW restrictions may be valid. For example, the title "Seed grower" (index = 561) can take on either "GOV," "OWN," or "PR" with industry code 0170. It is necessary to allow for all possibilities. 

Second, COW restrictions may apply to particular industry and not others. For example, the entry for "Flower planter" (index = 13114) has three different industry restrictions (0170, 0290, and 7770) but the COW restrictions of "GOV" or "PR" appear to only apply to industry code 0170. A simple rule to isolate batches of restrictions would to assume that restrictions are separated by commas and split the string `ind2017restr` on a comma to isolate each one. Then it is straightforward to evaluate restrictions as a group involving a industry and COW restrictions in combination. However, this strategy creates challenges with the "Seed grower" case, where splitting on commas would individually evaluate each COW but only the combination of industry code and COW with "PR 0170." In many cases throughout the index, commas are a reliable indicator of distinct restrictions. As of this writing, I exploit that variation but note that it might cause some errors. This is an area where index design changes could take the guesswork out of different patterns.

Third, and not shown in the examples above, is that some COW restrictions may be exceptions. In these cases, it is necessary to negate the COW restriction and, as before, the negated COW restriction may be in combination with an industry code. Such an example would be "Exc. LGOV 9570." In general, I treat negations as a separate class of restriction whereby I split on the exception indicator (e.g., "exc" or "exc.") and use the same functions to flag industry or COW restrictions as non-exceptions, then negate those strings. I will expand more on this in the section about industry restrictions.

Below I present the function to convert COW restrictions to a text string that Python can parse and evaluate for a given value of `COW` from the ACS.

In [12]:
# DEFINE FUNCTION TO RETURN COW RESTRICTIONS
def cowrestr( input_string ):
    # define dictionary of class of worker values
    cow_dict = { 'OWN' : [ 6, 7 ] ,
                 'GOV' : [ 3, 4, 5 ] ,
                 'PR' : [ 1, 2 ] ,
                 'OBNI' : 6 ,
                 'OBI' : 7 ,
                 'WP' : 8 ,
                 'LGOV' : 3 ,
                 'SGOV' : 4 ,
                 'FGOV' : 5 }
    
    # define regex from the keys of 'cow_dict'
    cowpat = '(?<!exc\.\s)(OWN|(?<![lsf])GOV|PR|OBNI|OBI|WP|LGOV|SGOV|FGOV)(?![a-z])'
    hascow = re.findall( cowpat , input_string , flags = re.I )
    
    # return condition from input string
    if len( hascow ) == 1:
        cow = hascow[ 0 ]
        cowval = cow_dict[ cow ]
        cowtype = isinstance( cow , (list) )
        if cowtype:
            return( '{{0}} in {}'.format( cowval ) )
        else:
            return( '{{0}} == {}'.format( cowval ) )
    elif len( hascow ) > 1:
        return( ' or '.join( ['{{0}} in {}'.format(cow_dict[x]) if isinstance(cow_dict[x],list) 
                              else '{{0}}=={}'.format(cow_dict[x]) for x in hascow]) )
    else:
        pass

Next, I present a sample of results form `cowrestr()` from the index. This is a simple case where no splitting on the comma occurs to separate distinct groupings, thus *all* potential COW restrictions are presented (see, for example, title "Lease operator" (index = 71)). As with education restrictions, the open field ("{0}") would be substituted with the value of `COW` for a matching write-in and `cowrestr` would be evaluated to ensure that any COW restrictions to be met.

In [15]:
cowdf['cowrestr'] = cowdf['ind2017restr'].apply(lambda x: cowrestr(x))
cowdf[['title','cenocc','ind2017restr','cowrestr']].head(20)

,title,cenocc,ind2017restr,cowrestr
16,City superintendent,0010,LGOV 9370,{0} == 3
28,Highway commissioner,0010,GOV 0770,"{0} == [3, 4, 5]"
46,Supervisor city,0010,LGOV 9370,{0} == 3
71,Lease operator,0020,"OWN 0370, PR 0370","{0} in [6, 7] or {0} in [1, 2]"
85,Oil lease operator,0020,OWN 0370,"{0} == [6, 7]"
86,Oil operator,0020,OWN 0370,"{0} == [6, 7]"
87,Oil producer,0020,OWN 0370,"{0} == [6, 7]"
89,Park superintendent,0020,GOV,"{0} == [3, 4, 5]"
90,Producer,0020,OWN 0370,"{0} == [6, 7]"
95,Shop superintendent,0020,"GOV, PR","{0} in [3, 4, 5] or {0} in [1, 2]"


#### Industry Restrictions
This section continue with our strategy of using `regex` to identify any industry code restrictions within the occupation index field `ind2017restr` to construct a string that takes an industry code from the ACS to evaluate whether a write-in meets all necessary conditions for being assigned an occupation code. As mentioned in the previous section, industry restrictions may come with COW restrictions in addition to exceptions where some industry codes are not allowed. In additions, some industry restrictions may include a range of industry codes. Ranges are separated by a hyphen ("-"), which serves as a useful pattern for indentification (it is also possible that ranges of industry codes may be excepted).

Industry restrictions also incorporate symbols to direct the assignment of *industry* codes from occupation write-ins. For example, if an industry write-in is missing but the respondent reports a job title matching the index, then industry code references wrapped in parentheses (e.g., "(0170)") should be assigned to the industry field. In addition, industry codes following a pound sign ("#") must be assigned to if a write-in and occupation code match the index record where such a pattern exists. For our purposes, we are using labeled data where `IND` was already assigned. As such, we treat these special cases as a valid industry restriction no different from an industry code with none of these symbols. Nonetheless, it would be straightforward to accommodate these rules into the functions that I develop here. (Note also that these patterns may also apply to COW fields.)

As mentioned in the previous section, each entry of `ind017restr` in the occupation index for which commas appear in the text string will be assumed to comprise multiple individual industry (and possibly COW) combinations. Unlike previous functions (`edrestrs()` and `cowrestr()`) that return a text string, this function returns a JSON object of restrictions, and each element within the JSON object is a set of industry and/or COW restriction(s) to be parsed and evaluated. I will demonstrate how to use this information at a later step, but I found this easier than constructing a complex batter of `regex` that could account for different combinations of restrictions. Indeed, this is not the only solution but the one that worked for me in the time given to implement this routine.

A JSON object is a list of dictionaries. Each element comprises a set of key-value pairs, where the keys (values) are: `cow` (conditional representation of a COW restriction for a given element), `code` (the industry code restriction), `range` (a range of industry codes), `text` (additional "titles within industry code" restrictions), and `exc` (a JSON object containing the same keys but the values are to be exceptions). In cases where no restrictions apply, the returned values are `None`.

The function `indrestr()` is presented below. It has a set of nested functions that determine the presence of exceptions. COW restrictions, range depictions, or additional text. After splitting on the comma to identify individual checks, it uses these rules to construct COW and industry-specific strings for each element that can be individually parsed by Python.

In [16]:
# FUNCTION TO RETURN INDUSTRY RESTRICTIONS
def indrestr( input_string ):
    # DEFINE INNER HELPER FUNCTIONS
    # return indicator of having an exception
    def hasexc( ind ):
        exc = re.findall( 'exc\.?' , ind , flags = re.I )
        if len( exc ) > 0:
            return 1
        else:
            return 0
    
    
    # return a numeric range (will require a numeric conversion at runtime)
    def indrange( ind , exc ):
        text = re.split( 'exc\.?' , ind , flags = re.I )[ exc ].strip()
        
        isrange = re.findall( '\d+\s?\-\s?\d+' , text )
        
        if len( isrange ) > 0:
            rangesplit = [ int(x.strip()) for x in re.split( '\s?-\s?' , isrange[0] ) ]
            return( 'int({{0}}) in range({0},{1}+1)'.format( * rangesplit ) )
        else:
            pass
    
    
    # return text component of industry restriction
    def indtext( ind , exc ):
        text = re.split( 'exc\.?' , ind , flags = re.I )[ exc ].strip()
        
        hastext = re.findall( '[a-z\.\-\'\s]+' , text , flags = re.I )
        cow_regex = '(OWN|(?<![lsf])GOV|PR|OBNI|OBI|WP|LGOV|SGOV|FGOV)(?![a-z])'
        repat = '\s?(((?<![a-z\d])\-(?![\da-z]))|((?<![a-z])n\.?s(?![a-z])\.?))\s?'
        
        if len( hastext ) > 0:
            # delete components of repat
            firstdel = re.sub( repat , '' , ''.join( hastext ).strip() , flags = re.I )
            if len( firstdel ) > 0:
                # make sure remaining components are not in cow_regex
                secdel = re.sub( cow_regex , '' , firstdel ).strip()
                # finally, remove stopwords
                nostop = [ x for x in word_tokenize( secdel ) if x.lower() not
                           in stopwords.words('english') ]
                if len( nostop ) > 0:
                    outtext = ' '.join( nostop )
                    # remove spaces before punctuation as a result of 'nostop'
                    return( re.sub( "\s(?=[\'\-\.])" , "" , outtext.strip() ) )
                else:
                    pass
            else:
                pass
        else:
            pass
    
    
    # return a negation if there is an EXCLUSION indicator
    def butnot( ind , exc ):
        if exc > 0:
            excsplit = re.split( 'exc\.?\s?' , ind , flags = re.I )
            excval = excsplit[ exc ].strip()
            
            # get code (if any)
            hasexccode = re.findall( '(?<!\-)\d{4}(?!\-)' , excval )
            
            if len( hasexccode ) == 1:
                exccode = '"{{0}}"=="{}"'.format( hasexccode[ 0 ] )
            elif len( hasexccode ) > 1:
                exccode = '"{{0}}" in {}'.format(["{}".format( x ) for x in hasexccode])
            else:
                exccode = None
            
            # exclusion is a range
            if indrange( excval , 0 ) is not None:
                #excrange = re.sub( '\s(?=in)' , ' not ', indrange( excval , 0 ) )
                excrange = indrange( excval , 0 )
            else:
                excrange = None
            
            # restriction has text component
            exctext = indtext( excval , 0 )
            
            # cow restriction
            exccow = cowrestr( excval )
            
            return( { 'code' : exccode , 
                      'range' : excrange ,
                      'text' : exctext ,
                      'cow' : exccow } )
        else:
            pass
    
    
    # return industry restrictions for a generic industry string
    def indcode( ind , exc ):
        repat = '(?<!\-)\s?\d{4}(?!\s?\-)'
        
        text = re.split( 'exc\.?' , ind , flags = re.I )[ exc ].strip()
            
        hascode = [x.strip() for x in list(set(re.findall(repat,text)))]
        
        if len( hascode ) == 1:
            return '"{{0}}"=="{}"'.format( hascode[ 0 ] )
        elif len( hascode ) > 1:
            return '"{{0}}" in {}'.format( ["{}".format( x ) for x in hascode] )
        else:
            pass
    
    
    # WORK WITH INPUT STRING
    # clean input_string
    # (1) remove noisy punctuation
    clean_pat = '([\(\)\#]+)|(\\\\\s?Any not listed)|(not listed above)'
    input_clean = re.sub( clean_pat , '' , input_string , flags = re.I ).strip()
    
    # (2) fix some common abbreviations
    # NOTE: define dictionary {key = regex of string, value = replacement}
    abbrev_dict = { 'depts?' : 'department' ,
                    'const' : 'construction' ,
                    'co' : '' ,
                    'mfg' : 'manufacturing' ,
                    'etc' : '' ,
                    'admin' : 'administration' ,
                    'n\.e\.c' : '' }
    
    for i in abbrev_dict.keys():
        input_clean = re.sub( '(?<![a-z\d]{{1}}){}\.?(?=\s|$)'.format( i ) ,
                              abbrev_dict[ i ] ,
                              input_clean ,
                              flags = re.I )
    
    # split on comma and work with industry restrictions
    # NOTE: multiple restrictions coded this way relatively consistently
    strsplit = [ x.strip() for x in input_clean.split( ',' ) ]
    
    
    if len( strsplit ) > 0:
        indr = []
        
        for j in strsplit:
            exclude = hasexc( j )
            
            textr = indtext( j , 0 )
            ranger = indrange( j , 0 )
            cowr = cowrestr( j )
            icoder = indcode( j , 0 )
            notr = butnot( j , exclude )
            
            indr.append( { 'range' : ranger ,
                           'cow' : cowr ,
                           'exc' : notr ,
                           'code' : icoder ,
                           'text' : textr } )
    else:
        indr = None
        
    # return result
    return indr

The function `indrestr()` presented above combines COW and industry restrictions to create a complete set of restrictions for a write-in (`cowrestr()` is called within the function). Below I present some singular cases to demonstrate its output.

First consider the industry retriction "Exc. LGOV 9570." Note that the only restrictions is an exception. Therefore, text strings for COW and industry code are reported in the "exc" key for this one-element JSON object. In both cases, it sets the value of each that should be excluded if True. Since no other restrictions are stated, the values for keys outside of "exc" are all `None`.

In [17]:
indrestr('Exc. LGOV 9570')

[{'range': None,
  'cow': None,
  'exc': {'code': '"{0}"=="9570"',
   'range': None,
   'text': None,
   'cow': '{0} == 3'},
  'code': None,
  'text': None}]

Next, consider the industry restrictions for title "Sales food products" for Census occupation code 4850, which involves a range of allowable industry codes and one exception. The function returns a non-null value for range; note also that it converts the occupation code from a string to an integer so that the `range()` function can be used. Also note that the range adds the integer "1" to the upper boundary of the range specified in the industry restriction. This is necessary so that Python will include the upper boundary in the range. Since the only exception is one industry code (1190), that value appears in the "code" key within the "exc" value.

In [18]:
occdf[(occdf.title=='Sales food products') & (occdf.cenocc=='4850')][['title','cenocc','ind2017restr']]

,title,cenocc,ind2017restr
10377,Sales food products,4850,1070-4590 exc. 1190


In [19]:
indrestr('1070-4590 exc. 1190')

[{'range': 'int({0}) in range(1070,4590+1)',
  'cow': None,
  'exc': {'code': '"{0}"=="1190"', 'range': None, 'text': None, 'cow': None},
  'code': None,
  'text': None}]

In [20]:
indrestr('Fire or police department 9470')

[{'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="9470"',
  'text': 'Fire police department'}]

Next, consider the entry for "Sales computer software" for Census occupation code 4850. This restriction includes a range of allowable industry codes plus a comma-separated list of others. The function returns a list equal to the number of elements separated by commas. Note that the first one contains non-null values for the "range" key, while the other restrictions have non-null values for the "code" key. In each case, the non-values are presented as a string with an open field for string substitution, which will take a value of `IND` from the ACS record in which a write-in is "Sales computer software" that Python can parse and evaluate directly.

In [25]:
occdf[(occdf.title=='Sales computer software') & (occdf.cenocc=='4850')]

,job_title,ind2017restr,cenocc,soc2018,hased,title,edrestr
10360,Sales computer software,"1070-4590, 6490, 6690, 6695, 7380",4850,41-4010,0,Sales computer software,None


In [26]:
indrestr(occdf.iloc[10360,1])

[{'range': 'int({0}) in range(1070,4590+1)',
  'cow': None,
  'exc': None,
  'code': None,
  'text': None},
 {'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="6490"',
  'text': None},
 {'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="6690"',
  'text': None},
 {'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="6695"',
  'text': None},
 {'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="7380"',
  'text': None}]

Finally, I present an example that incorporates text elements in an industry restriction. The function is able to isolate the the industry code from the text string and present each as elements to be checked. While it is not implemented here, a production version of this function would want to also make the pareseable text string incorporte a `regex` that can account for write-ins that may place periods in between "U.S." as well as those who write "US." Note also that I have been using case-sensitivity to match that of the index presented thus far. When operationalized, the text strings should have their case matched to that of the ACS file.

In [25]:
indrestr('U.S. Custom House 9380')

[{'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="9380"',
  'text': 'U.S. Custom House'}]

In [21]:
# execute indrestr() on the index file and show the first 20 records
occdf['indrestr'] = np.where( occdf.ind2017restr.isna() ,
                              None ,
                              occdf.ind2017restr.apply(lambda x: indrestr(str(x))) )

occdf[['title','cenocc','ind2017restr','indrestr','edrestr']].head(20)

,title,cenocc,ind2017restr,indrestr,edrestr
0,Admiral,0010,NaN,None,None
1,Board chairman,0010,NaN,None,None
2,Board member,0010,NaN,None,None
3,Bureau chief,0010,9370-9590,"[{'range': 'int({0}) in range(9370,9590+1)', '...",None
4,CEO,0010,NaN,None,None
5,CFO,0010,NaN,None,None
6,Chairman of the board,0010,NaN,None,None
7,Chancellor,0010,7870,"[{'range': None, 'cow': None, 'exc': None, 'co...",None
8,Chief executive officer,0010,NaN,None,None
9,Chief financial officer,0010,NaN,None,None


The above shows the occupation index cleaned, pre-processed, and ready to implement with real ACS write-in data. The final step is define two sets of functions that operationalize the index with write-in data. The first is a set of two functions---`resolveed()` and `resolveind()`---that can use the index fields `edrestr` and `indrestr`, respectively, with ACS data to return indicators that a given education or industry restriction is satisfied. The second, `assignocc()`, takes these results and returns the occupation code(s) for a matching write-in (if any).

Before we present these final function and operationalize the index autocoder, let's save the pre-processed index for later use. Doing so is only necessary when the occupation index changes or if new rules are applied to the previous functions that alter the parseable strings associated with a given restriction. I use the JSON format because it is easy to view in a text editor, can be easily imported into a Python `pandas` DataFrame object, and occupies little space on disk.

In [22]:
# first convert dataframe to dictionary
occindex_dict = occdf[['title','cenocc','edrestr','indrestr']].to_dict(orient='records')

# export to json file
with open( 'occ_index_clean.json' , 'w' ) as f:
    json.dump( occindex_dict , f , indent = 4 )

### Putting it Together
This section develops the complete algorithm to match Census occupation codes to ACS write-ins using the occupation index. I combine output from the `edrestr()`, `cowrestr()`, and `indrestr()` functions applied to index entries and derive a set of decision rules to determine whether a write-in matching an index entry can take its associated occupation code (`cenocc`). Note that, as of this writing, we do *not* incorporate the text fields for restrictions. Further development is required.

Index entries with an education restriction apply broadly to a matching write-in. If it is not satisfied then the occupation code for that write-in is not allowed regardless of whether any industry or COW values are valid. Any matching write-in with no explicit education restriction is considered a valid candidate.

The previous section demonstrated the complexities of controlling for industry restrictions. The general strategy is to allow a candidate write-in to be associated with its occupation code from the index if any of the industry restrictions are true and all exceptions are met.

I operationalize the autocoder with user-defined functions that check for and evaluate the parseable functions created in the previous sections. `resolveed()` takes the value of `edrestr()` and returns one of `True`, `False`, or `None`. `resolveind()` resolves all potential industry and COW restrictions and exceptions into a single value of `True`, `False`, or `None`. Note that, while entries with no restrictions (as indicated by `None`) are valid candidates, I distinguish them from `True` so that they are only applied if no other `True` candidate is found. This allows for dealing with cases where the index specifies "Any" or "not elsewhere specified." Each function takes as its first parameter the result of `edrestr()` and `indrestr()`, respectively. Other paramters are the actual values from the ACS, which are used by each function to substitute into the open field ("{0}") and then parse and evaluate the string.

In [106]:
# define function to resolve education restrictions
# NOTES: resolves to (True, False, None). input parameters are:
#        edr = text string containing education reference in parseable code
#        ed  = value of years of schooling variable in ACS (e.g., SCHL)
def resolveed( edr , ed ):
    if edr is None:
        return None
    return eval( edr.format( ed ) )

In [108]:
# define function to resolve industry restrictions
# NOTES: resolves to (True, False). input paramters are:
#        indr = JSON object of industry/cow restrictions and exceptions
#        ind  = ACS variable for industry (e.g., IND); expecting a string
#        cow  = ACS variable for class of worker (e.g., COW)
def resolveind( indr , ind , cow ):
    if indr is None:
        return None
    else:
        # define a checks dictionary
        check_vals = { 'code' : ind ,
                       'range' : int(ind) ,
                       'cow' : cow }

        # initialize a results list
        ind_check = []

        # loop over elements of 'indr' (i.e., the json object)
        # NOTE: we exclude the 'text' key; this should be changed in production
        for i in indr:
            checks = [ x for x in i.keys() if i[x] is not None and x != 'text' ]

            # if 'text' is the only restriction, then consider False. if no restrictions, then True
            # NOTE: this is because we omit 'text'; treating it as 'False' here prevents a false positive
            #       though might induce a false negative. one method to incorporate the 'text' field is
            #       to add a parameter that takes the value of 'inw2' from the acs
            if len( checks ) == 0:
                if i['text'] is None:
                    ind_check.append( True )
                else:
                    ind_check.append( False )
            else:
                check_dict = {}

                for j in checks:
                    if j != 'exc':
                        check_dict[j] = eval( i[j].format( check_vals[j] ) )
                    else:
                        exc_keys = [ x for x in i[j].keys() if not i[j][x] is None and x != 'text' ]

                        if len( exc_keys ) == 0:
                            if i[j]['text'] is None:
                                check_dict[j] = True
                            else:
                                check_dict[j] = False
                        else:
                            exc_dict = {}
                            for k in exc_keys:
                                exc_dict[k] = eval( i[j][k].format( check_vals[k] ) )
                            if sum( exc_dict.values() ) == len( exc_keys ):
                                check_dict[j] = True
                            else:
                                check_dict[j] = False

                # return True only if all components are met, False otherwise
                if sum( check_dict.values() ) == len( checks ):
                    ind_check.append( True )
                else:
                    ind_check.append( False )

        # final industry restriction determination
        # NOTE: return True if any industry restriction is true, False otherwise
        if True in ind_check:
            return True
        else:
            return False

In [95]:
# working example
test_indr = occdf[occdf.title=='Concessionaire'.upper()].iloc[0][['indrestr']]
test_indr

indrestr    [{'range': None, 'cow': None, 'exc': {'code': ...
Name: 702, dtype: object

In [63]:
[y for y in [x for x in test_indr[0]][0] if test_indr[0][0][y] is not None]

['exc']

In [64]:
test_indr[0][0]

{'range': None,
 'cow': None,
 'exc': {'code': '"{0}"=="6570"', 'range': None, 'text': None, 'cow': None},
 'code': None,
 'text': None}

In [27]:
occdf[occdf.title=='Concessionaire']

,job_title,ind2017restr,cenocc,soc2018,hased,title,edrestr,indrestr
702,Concessionaire,"Exc. 6570, 8680",0310,11-9051,0,Concessionaire,None,"[{'range': None, 'cow': None, 'exc': {'code': ..."
8203,Concessionaire,"6570, 8680",4055,35-3023,0,Concessionaire,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."


In [79]:
sum([True, False])

1

In [66]:
resolveind(test_indr[0],'6750',1)

False

In [67]:
test_indr = occdf[occdf.title=='Teacher heat engineering'].iloc[0]['indrestr']
test_indr
resolveind(test_indr,'7880',None)

True

Finally, I specify a function that returns an overall indicator of whether an ACS write-in matching an occupation index title can be assigned the occupation code associated with that title. The function `assignocc()` combines the results from evaluating any education, industry, or COW restrictions and returns a single `True`, `False`, or `None` for that index entry. Values of `True` indicate the a write-in matching the index title has met all conditions for being assigned the Census occupation code (`cenocc`) from the index. A value of `False` means that the index entry is not valid occupation code assignment for an ACS write-in with a matching job title. A return value of `None` indicates that the matching job title was found to have no restrictions and is valid; later I will show a method for choosing amongst candidate values.

In [119]:
# define function to denote a valid match
def assignocc(edres = None, indres = None):
    if indres is None:
        if edres is None:
            return None
        else:
            if edres is True:
                return True
            else:
                return False
    elif indres is True:
        if edres is None:
            return True
        elif edres is True:
            return True
        else:
            return False
    else:
        return False

### Implementing the Exact Match Autocoder
This section implements the exact match autocoder on the internal 2018 and 2019 ACS 1-year samples. We restrict the sample to those with non-missing data for `OCC`, `ocw1`, `ocw2`, `indw2`, `indw3`, `SCHL`, `COW`, and `IND` since they are the required inputs. I merge the clean occupation index with the ACS data on index `title` and evaluate `resolveed()` and `resolveind()` for matching cases using input from the ACS file. I then apply the `assignocc()` function to the output of `resolved()` and `resolveind()` to flag matching job titles as potential autocode matches from the occupation index.

I found that the fastest way to implement the autocoder is to perform a merge on job titles then use a `lambda` function to evaluate any restrictions for the set of matches. I attempt to match index titles on three combinations of the ACS occupation write-in fields: `ocw1`, `ocw2`, and the joint `ocw1` and `ocw2`. For the latter case, I construct a composite string which is simply the cocatenation of both fields separated by a space (" "). Some index titles have multiple entries, which opens up the possibility of many possible matches for a given write-in. Interestingly, I find index job titles that match for each specification of write-in values. This suggest that the occupation index can be used as multi-stage operation for finding matches prior to relying on stochastic methods.

One feature of the occupation index is that some job titles have multiple entries. In most cases (see the entry for "Concessionaire" in the previous section), these entries are mutually exclusive with the addition of specific education, industry, or COW restrictions. The functions developed in the previous section are critical in choosing an occupation code based on other information provided by the respondent. In addition, the many possible index title matches for a given write-in produces a many-to-one match that results in a relatively large file (in excess of 12 million records as I show below). However, operating the functions developed here on a large file is easily handled within the IRE and produces a result faster than many ML models on a scaled-down set of observations. I incorporate some timing functions to demonstrate the time it takes to operate on these files.

In what follows, I first link the ACS write-in fields to the occupation index developed in the previous section. I construct a unique index for each record in the ACS so that I can both demonstrate the many-to-one matches that may occur because of duplication of job titles in the index, as well as to match the result of `assignocc()` to the original file so that I can compare the peformance of the exact match to the current logistic regression autocoder. Second, I execute the `ressolveed()` and `resolveind()` functions for each write-in before executing the `assignocc()` function. Along the way I present examples from the write-in with original industry restrictions content from the occupation index so that you can see how the system works. Third, I specify a routine that selects the matching autocode occupation for a given write-in. This routine omits `False` index entries but also chooses `None` only if no `True` are present. If multiple occupation codes are considered matches, I store all of them in a list.

#### Linking the Index to the ACS Write-In Fields
Here we link the ACS write-in fields to the occupation index restrictions developed earlier in this notebook. Doing so requires three separate merges. I must also create the composite of both write-ins, which I call `ocw3`. In addition, we re-cast the index titles to uppercase to match the format of ACS write-ins.

In [98]:
# import time library
import time

# define set of keep variables from the acs
acs_keep = ['OCC','auto_uocc','ocw1','ocw2','IND','inw2','inw3','SCHL','COW']
occ_keep = ['title','cenocc','edrestr','indrestr','ind2017restr']

# set occ index titles to uppercase
occdf['title'] = occdf.title.apply(lambda x: x.upper())

# create a sequence variable identifying each record as unique
acs['recid']=np.arange(len(acs)).astype(str)

# remove trailing and leading spaces (if any) from ocw1 and ocw2
acs['ocw1'] = acs.ocw1.apply(lambda x: x.strip())
acs['ocw2'] = acs.ocw2.apply(lambda x: x.strip())

# create composite of ocw1 and ocw2
acs['ocw3'] = acs.ocw1.apply(lambda x: str(x).strip()) + ' ' + acs.ocw2.apply(lambda x: str(x).strip())


# merge
acs_keep.append('recid')

st_time = time.time()
acsmrg1 = pd.merge(acs[acs_keep], 
                   occdf, 
                   left_on='ocw1', 
                   right_on='title')
en_time = time.time()

print( 'time to complete merge on ocw1:' , '%.2f' % (en_time - st_time) , 'seconds')

st_time = time.time()
acsmrg2 = pd.merge(acs[acs_keep], 
                   occdf, 
                   left_on='ocw2', 
                   right_on='title')
en_time = time.time()

print( 'time to complete merge on ocw2:' , '%.2f' % (en_time - st_time) , 'seconds')


acs_keep3 = acs_keep.copy()
acs_keep3.append( 'ocw3' )

st_time = time.time()
acsmrg3 = pd.merge(acs[acs_keep3 ], 
                   occdf, 
                   left_on='ocw3', 
                   right_on='title')
en_time = time.time()

print( 'time to complete merge on ocw3:' , '%.2f' % (en_time - st_time) , 'seconds')

time to complete merge on ocw1: 17.62 seconds
time to complete merge on ocw2: 5.31 seconds
time to complete merge on ocw3: 4.25 seconds


The occupation index is able to match titles for each of these fields. I will examine whether these matches are unique in a moment. First, we need to resolve the industry and education restrictions for each of these data sets. Because these functions use multiple paramters, we move away from the `apply(lambda...)` approach and instead use the `vectorize()` function from the `numpy` library.

In [113]:
# resolve industry and education fields using the resolveed and resolveind functions
st_time = time.time()
acsmrg1['resed'] = np.vectorize(resolveed)(acsmrg1['edrestr'], acsmrg1['SCHL'])
en_time = time.time()
resed1_time = en_time - st_time

st_time = time.time()
acsmrg2['resed'] = np.vectorize(resolveed)(acsmrg2['edrestr'], acsmrg2['SCHL'])
en_time = time.time()
resed2_time = en_time - st_time

st_time = time.time()
acsmrg3['resed'] = np.vectorize(resolveed)(acsmrg3['edrestr'], acsmrg3['SCHL'])
en_time = time.time()
resed3_time = en_time - st_time

st_time = time.time()
acsmrg1['resind'] = np.where(acsmrg1.indrestr.isnull() ,
                             None,
                             np.vectorize(resolveind)(acsmrg1['indrestr'], acsmrg1['IND'], acsmrg1['COW']))
en_time = time.time()
resind1_time = en_time - st_time

st_time = time.time()
acsmrg2['resind'] = np.where(acsmrg2.indrestr.isnull(),
                             None,
                             np.vectorize(resolveind)(acsmrg2['indrestr'], acsmrg2['IND'], acsmrg2['COW']))
en_time = time.time()
resind2_time = en_time - st_time

st_time = time.time()
acsmrg3['resind'] = np.where(acsmrg3.indrestr.isnull(),
                             None,
                             np.vectorize(resolveind)(acsmrg3['indrestr'], acsmrg3['IND'], acsmrg3['COW']))
en_time = time.time()
resind3_time = en_time - st_time

In [120]:
# determine whether a matching index entry can be assigned to a write-in
st_time = time.time()
acsmrg1['assignocc'] = np.where(acsmrg1.resed.isnull() & acsmrg1.resind.isnull(),
                                None,
                                np.vectorize(assignocc)(acsmrg1['resed'], acsmrg1['resind']))
en_time = time.time()
occ1_time = en_time - st_time

st_time = time.time()
acsmrg2['assignocc'] = np.where(acsmrg2.resed.isnull() & acsmrg2.resind.isnull(),
                                None,
                                np.vectorize(assignocc)(acsmrg2['resed'], acsmrg2['resind']))
en_time = time.time()
occ2_time = en_time - st_time

st_time = time.time()
acsmrg3['assignocc'] = np.where(acsmrg3.resed.isnull() & acsmrg3.resind.isnull(),
                                None,
                                np.vectorize(assignocc)(acsmrg3['resed'], acsmrg3['resind']))
en_time = time.time()
occ3_time = en_time - st_time

#### Merge Result on `ocw1`
This subsection presents a high-level summary of merging the occupation index to ACS write-ins based on index `title` and ACS variable `ocw1`. The merge results in a file with over 12.5 million records. It took just under five minutes to execute the functions needed to generate an autocode.

I also print the first 20 records of the resulting merge with evaluation of each candidate index occupation title. Presented reults are for the job title "LABORER." The variable `recid` corresponds to a particular record in the ACS file. Note that the index has 10 entries with job title "LABORER" with all but two having an explicit industry restriction. No entry has an education restriction, which is denoted by the value `resed` = `None`. The variable `resind` presents the result of the `resolveind()` function. The two respondents listed in the example have values of `IND` equal to 7580, which does not satisfy the restrictions outlined in `ind2017restr` and, as a result, have values of `resind` equal to False. The only `True` results are based on the "Any not listed" entries that are associated with `cenocc` 9620, which was the value ultimately assigned to the data (see variable `OCC`). Note also that the existing logistic regression autocoder was unable to make a prediction for this case (i.e., `auto_uocc` = `NaN`). When choosing an occupation code for this example, we would rule out any `False` cases and keep only the occupation codes associated with `True` (or `None` if not `True` are available). In this case, both `True` are the same occupation code and we would simply drop the duplicate.

This example does demonstrate one possible error in the index and evaluation functions developed thus far. The two matching occupations codes (i.e., `assignocc` = `True`), there is no indusry restriction. As such, `resind` should resolve to `None`. This would not affect the choice of industry autocode in this example because all other possibilities are explicitly rejectd as `False`. However, this could pose a problem in cases where a mix of `None` and `True` should be the result but both all non-`False` are equal to `True`. In the former, we could select the `True`, while in the latter we may be stuck with two potential matches. Additional work with more time could fix this issue.

In [122]:
# look at dimension of merged file
print( 'shape of merged ACS file with index on ocw1:' , acsmrg1.shape )

# present the total time to come up with an autocode result
print('total time to select occupation code by ocw1:' ,
      '%.2f' % (resed1_time + resind1_time + occ1_time ) , 'seconds' )

# print the first 20 records 
acsmrg1[['recid','OCC','auto_uocc','ocw1','title','edrestr','ind2017restr',
         'cenocc','COW','SCHL','IND','resed','resind','assignocc']].head(20)

shape of merged ACS file with index on ocw1: (12585382, 21)
total time to select occupation code by ocw1: 263.20 seconds


,recid,OCC,auto_uocc,ocw1,title,edrestr,ind2017restr,cenocc,COW,SCHL,IND,resed,resind,assignocc
0,1,9620,NaN,LABORER,LABORER,None,"7770, Cemetery 7072, 9080 or 9480, Golf course...",4251,1,15,7580,None,False,False
1,1,9620,NaN,LABORER,LABORER,None,"0170, 0180, 0290",6050,1,15,7580,None,False,False
2,1,9620,NaN,LABORER,LABORER,None,0190,6120,1,15,7580,None,False,False
3,1,9620,NaN,LABORER,LABORER,None,0270,6130,1,15,7580,None,False,False
4,1,9620,NaN,LABORER,LABORER,None,0770,6260,1,15,7580,None,False,False
5,1,9620,NaN,LABORER,LABORER,None,0370-0490,6950,1,15,7580,None,False,False
6,1,9620,NaN,LABORER,LABORER,None,"5090, 8770, 8780",7610,1,15,7580,None,False,False
7,1,9620,NaN,LABORER,LABORER,None,"6080, 6180, 6280",9620,1,15,7580,None,False,False
8,1,9620,NaN,LABORER,LABORER,None,NaN,9620,1,15,7580,None,None,None
9,1,9620,NaN,LABORER,LABORER,None,\ Any not listed,9620,1,15,7580,None,True,True


#### Merge Result on `ocw2`
Next I present a summary of results merging index job titles on the ACS write-in associated with `ocw2`. Despite the ACS field `ocw1` being intended for job titles and `ocw2` being reserved for a description of duties, the latter includes job titles that match the occupation index. The resulting file includes just under 3.5 million records (these could be many index matches for a single ACS write-in) and, unsurprisingly, it takes less time to execute the occupation autocoder functions on the file (just under 2 minutes).

Example matches for this case are for the index occupation "PATIENT CARE," which has no indutry restriction but an education component. Several important features of the exact match autocoder used here emerge. First, a single occupation code can be matched for all five respondents. The existing autocoder was unable to provide a match for two of them (`recid` = (17, 135)). Second, the existing autocoder produced codes that are inconsistent with the index. For example, the existing autocode predicted for `recid` = 136 an occupation code of 3603, which was ultimately assigned as the final output value. However, this respondent has a value of `SCHL` = 19, which suggests it should be assigned a code of 3500 based on `assignocc`. Indeed, the processes that generate a value of `auto_uocc` are more involved than those used by `assignocc`, but this demonstrates a case where stochastic methods inherently have probabilistic error that a deterministic autocoder like that generated from the index does not.

In [121]:
# look at dimension of merged file
print( 'shape of merged ACS file with index on ocw2:' , acsmrg2.shape )

# present the total time to come up with an autocode result
print('total time to select occupation code by ocw2:' ,
      '%.2f' % (resed2_time + resind2_time + occ2_time ) , 'seconds' )

# print the first 20 records 
acsmrg2[['recid','OCC','auto_uocc','ocw2','title','edrestr','ind2017restr',
         'cenocc','COW','SCHL','IND','resed','resind','assignocc']].head(20)

shape of merged ACS file with index on ocw2: (3596732, 21)
total time to select occupation code by ocw2: 100.41 seconds


,recid,OCC,auto_uocc,ocw2,title,edrestr,ind2017restr,cenocc,COW,SCHL,IND,resed,resind,assignocc
0,17,3500,NaN,PATIENT CARE,PATIENT CARE,{0}==23,NaN,3090,1,19,8180,False,None,False
1,17,3500,NaN,PATIENT CARE,PATIENT CARE,{0}==20 or {0}==21 or {0}==22,NaN,3255,1,19,8180,False,None,False
2,17,3500,NaN,PATIENT CARE,PATIENT CARE,"{0} in [16, 17] or {0} in [18, 19]",NaN,3500,1,19,8180,True,None,True
3,17,3500,NaN,PATIENT CARE,PATIENT CARE,{0}<16,NaN,3603,1,19,8180,False,None,False
4,32,3255,3255,PATIENT CARE,PATIENT CARE,{0}==23,NaN,3090,1,21,8191,False,None,False
5,32,3255,3255,PATIENT CARE,PATIENT CARE,{0}==20 or {0}==21 or {0}==22,NaN,3255,1,21,8191,True,None,True
6,32,3255,3255,PATIENT CARE,PATIENT CARE,"{0} in [16, 17] or {0} in [18, 19]",NaN,3500,1,21,8191,False,None,False
7,32,3255,3255,PATIENT CARE,PATIENT CARE,{0}<16,NaN,3603,1,21,8191,False,None,False
8,75,3258,3258,PATIENT CARE,PATIENT CARE,{0}==23,NaN,3090,1,22,8191,False,None,False
9,75,3258,3258,PATIENT CARE,PATIENT CARE,{0}==20 or {0}==21 or {0}==22,NaN,3255,1,22,8191,True,None,True


#### Merge Result on `ocw3`
This subsection presents the results of merging index job titles with a composite string of ACS fields `ocw1` and `ocw3`. The key result is that index job titles do match with this composite variable. In the examples provided below, `assignocc` produces values of `None` for each case but, since there are no `True` or `False` matches, these cased could be assigned the value of `cenocc` from the index. Note that in the example cases, the existing autocoder was unable to make predictions for these cases but the job index can. In addition, the predictions from the job index equals the occupation code that was ultimately assigned.

It is worth noting that these same cases may have been matched on `ocw1` for "LABORER." We can design a method that matches each write-in (plus the composite `ocw3`) with the index and iteratively selects a single occupation code at different stages. For example, if an ambiguous match is found using `ocw1`, no match was found on `ocw2`, and a non-ambiguous match on `ocw3`, we can assign the code based on `ocw3` and assign a match "code" to the record so that analysts will have a record of how the exact match autocoder made its assignment. We turn to that in the next section.

In [123]:
# look at dimension of merged file
print( 'shape of merged ACS file with index on ocw3:' , acsmrg3.shape )

# present the total time to come up with an autocode result
print('total time to select occupation code by ocw3:' ,
      '%.2f' % (resed3_time + resind3_time + occ3_time ) , 'seconds' )

# print the first 20 records 
acsmrg3[['recid','OCC','auto_uocc','ocw3','title','edrestr','ind2017restr',
         'cenocc','COW','SCHL','IND','resed','resind','assignocc']].head(20)

shape of merged ACS file with index on ocw3: (38364, 22)
total time to select occupation code by ocw3: 0.51 seconds


,recid,OCC,auto_uocc,ocw3,title,edrestr,ind2017restr,cenocc,COW,SCHL,IND,resed,resind,assignocc
0,1,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,15,7580,None,None,None
1,503875,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,16,4270,None,None,None
2,599492,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,16,3570,None,None,None
3,676355,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,14,2170,None,None,None
4,732827,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,19,3570,None,None,None
5,974493,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,16,7580,None,None,None
6,990755,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,11,7580,None,None,None
7,1021341,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,19,1370,None,None,None
8,1121910,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,16,3291,None,None,None
9,1148349,9620,NaN,LABORER WAREHOUSE,LABORER WAREHOUSE,None,NaN,9620,1,16,6390,None,None,None


#### Collapsing Result of `assignocc` to a Single Record
Before we move on to evaluating the result of the exact match autocoder, it is necessary to collapse valid occupation codes from the set of matches to a single record. This will enable merging back with the original ACS data so that we can compare the prediction rate and quality to `OCC` and `auto_uocc`. Here, I apply the same strategy for each of the three merge files generated in this section, then merge each with the original ACS file. For each set of merges, I first delete instances of `assignocc` = `False`. Second, I aggregate the unique values of `cenocc` by `recid` for (a) cases where `assignocc` = `True` and (b) `assignocc` = `None`. Third, I compare the result of `True` and `None`. I prioritize code assignments with values of `True` but default to `None` of no `True` are found.

In [124]:
# collapse cenocc into space-delmited string for cases where assignocc = True
acstrue1 = acsmrg1[acsmrg1.assignocc==True][['recid','cenocc']].astype(str).groupby('recid').agg(lambda x: ' '.join( x.unique())).copy().reset_index()
acstrue2 = acsmrg2[acsmrg2.assignocc==True][['recid','cenocc']].astype(str).groupby('recid').agg(lambda x: ' '.join( x.unique())).copy().reset_index()
acstrue3 = acsmrg3[acsmrg3.assignocc==True][['recid','cenocc']].astype(str).groupby('recid').agg(lambda x: ' '.join( x.unique())).copy().reset_index()

In [125]:
# collapse cenocc into space-delmited string for cases where assignocc = None
acsnone1 = acsmrg1[acsmrg1['assignocc'].isnull()][['recid','cenocc']].astype(str).groupby('recid').agg(lambda x: ' '.join( x.unique())).copy().reset_index()
acsnone2 = acsmrg2[acsmrg2['assignocc'].isnull()][['recid','cenocc']].astype(str).groupby('recid').agg(lambda x: ' '.join( x.unique())).copy().reset_index()
acsnone3 = acsmrg3[acsmrg3['assignocc'].isnull()][['recid','cenocc']].astype(str).groupby('recid').agg(lambda x: ' '.join( x.unique())).copy().reset_index()

In [126]:
# rename 'cenocc' column in each data frame
for i in range(1, 4):
    for j in ['true', 'none']:
        tabname = 'acs{0}{1}'.format(j, i)
        occname = 'occ{0}{1}'.format(i, j)
        globals()[tabname].rename(columns={'cenocc' : occname}, inplace=True)

In [127]:
# merge results with original acs file
dat_list = ['acs']

for i in range(1, 4):
    for j in ['true', 'none']:
        tabname = 'acs{0}{1}'.format(j, i)
        if len( globals()[tabname] ) > 0:
            dat_list.append( tabname )

dat_list

['acs', 'acstrue1', 'acsnone1', 'acstrue2', 'acstrue3', 'acsnone3']

In [128]:
from functools import reduce

acs_mrg = reduce( lambda left, right: pd.merge(left, right, on=['recid'], how='outer'),
                                               [globals()[x] for x in dat_list] )

In [129]:
# check the shape of the resulting merge file
print( 'original acs file has same number of records as the merged file:', len(acs) == len(acs_mrg) )
print( 'shape of resulting merge file:' , acs_mrg.shape )

original acs file has same number of records as the merged file: True
shape of resulting merge file: (4533644, 84)


In [130]:
acs_mrg.columns

Index(['CMID', 'PNUM', 'CTY', 'ST', 'GQ', 'IMO', 'MODE', 'UAGE', 'AGE', 'FAGE',
       'CIT', 'UCOW', 'COW', 'FCOW', 'ESR', 'FESR', 'FOD1', 'FOD10', 'FOD2',
       'FOD3', 'FOD4', 'FOD5', 'FOD6', 'FOD7', 'FOD8', 'FOD9', 'UIND', 'IND',
       'FIND', 'UJWTR', 'JWTR', 'UMIL', 'MIL', 'FMIL', 'UOCC', 'OCC', 'FOCC',
       'USCHL', 'SCHL', 'USEM', 'SEM', 'FSEM', 'SEX', 'UWAG', 'WAG', 'FWAG',
       'UWKH', 'WKH', 'UWKL', 'WKL', 'FWKL', 'UWKW', 'WKW', 'FWKW', 'UWKWX',
       'UWRK', 'UWRKJ', 'WRK', 'HSGP', 'PAOC', 'XCOW', 'XIND', 'XOCC', 'seq',
       'dcf_ucow', 'inw2', 'inw3', 'inx4', 'ocw1', 'ocw2', 'tocoding', 'INX1',
       'auto_uind', 'auto_uocc', 'GQMAJTYP', 'year', 'SEQ', 'recid', 'ocw3',
       'occ1true', 'occ1none', 'occ2true', 'occ3true', 'occ3none'],
      dtype='object')

In [164]:
# look at first few records
acs_mrg[['OCC','auto_uocc','IND','ocw1','ocw2','occ1true','occ1none','occ2true','occ3true','occ3none']].head(20)

,OCC,auto_uocc,IND,ocw1,ocw2,occ1true,occ1none,occ2true,occ3true,occ3none
0,4220,NaN,7072,CONDO CLEANING,CLEANING CONDOS,NaN,NaN,NaN,NaN,NaN
1,9620,NaN,7580,LABORER,WAREHOUSE,9620,9620,NaN,NaN,9620
2,4220,NaN,5381,MAINTENANCE,STOCKING/CLEANING,4220,NaN,NaN,NaN,NaN
3,9130,9130,0770,DRIVING A TRUCK,DRIVING THE TRUCK WHILE STRIPING THE ROAD,NaN,NaN,NaN,NaN,NaN
4,3422,NaN,8090,BEHAVIORAL AIDE,TEACHING MENTAL PATIENTS BASIC SKILLS,NaN,NaN,NaN,NaN,NaN
5,7260,NaN,5391,TIRE/BATTERY SPECIALIST,CHANGING TIRES & BATTERIES,NaN,NaN,NaN,NaN,NaN
6,4621,4621,8590,PERSONAL TRAINING MANAGER,CLEARICAL WORK SHOWING MEMBERS ROUTINES AND EX...,NaN,NaN,NaN,NaN,NaN
7,4110,NaN,8680,SERVING FOOD AND MAKING COCKTAILS,SERVING FOOD AND MAKING COCKTAILS,NaN,NaN,NaN,NaN,NaN
8,7340,NaN,9370,MAINT,REPAIRS,NaN,NaN,NaN,NaN,NaN
9,0440,NaN,8590,MANAGER,MANAGING,0335 0440,NaN,NaN,NaN,NaN


#### Making Final Code Assignment
Time constraints prevent a thorough analysis of occupation index assignment at different levels of ACS write-ins (e.g., `ocw1`, `ocw2`, `ocw3`). For example, it is possible to compare the code assignments between one another, as well as comparing each with value of `OCC` that was ultimately assigned in these test data. Instead, I develop a simple rule to determine a match based on a single result from the index autocoder developed in this notebook.

The strategy is as follows. I iteratively work from matches based on `ocw1` through `ocw3`. At each stage, if a single matched occupation code is matched for the `True` case (e.g., `occ1true`) then I take that and stop. If there is no match, I move to the `None` case (e.g., `occ1none`). If a match is found there I take that and stop. I repeat the process through `occ3none` until a match is found. If no match is found then I set the index autocode value to `None`.

One deviation from this process is in the case of a list result where more than one index autocode is found at a given stage. If a list is found, then I continue the process until only a single value is found. If no other single value is found, then I retain the first list with a match and conclude that the index autocoder was able to find a set of potential matches, but was unable to isolate one or the other.

Along with each index autocode output, I record a three-digit numeric code that indicates when a match was made. This code will enable analysts to summarize how the index autocoder is making matches and also to isolate certain interesting cases for further exploration *ex post*. The three-code is defined as follows: the first digit takes on a value of 1, 2, or 3 corresponding to the `ocw` write-in field from which it was derived; the second digit equals 0 if the match was taken from the `None` field and a 1 indicates it was taken from a `True` field; the last digit takes a value of 1 or 2, indicating the result is a single code or from a list, respectively.

Next, I specify a final function to assign the index autocode value and its match code. The output is a dictionary and one can call a specific key from that dictionary at execution time.

In [155]:
# function to output index autocode
def indexautocode(occ1t, occ1n, occ2t, occ3t, occ3n, occ2n=None):
    occ_out = ''
    match_level = 0
    
    for i, j in zip([occ1t, occ1n, occ2t, occ2n, occ3t, occ3n],
                    [11, 10, 21, 20, 31, 30]):
        if not pd.isnull(i):
            occ_out = i
            match_level = j
            if len(occ_out.split()) == 1:
                break
    
    if match_level == 0:
        return {'indexocc' : None ,
                'match_code' : None }
    else:
        if len(occ_out.split()) > 1:
            match_level = match_level * 10 + 2
        else:
            match_level = match_level * 10 + 1
        return {'indexocc' : occ_out ,
                'match_code' : match_level}

In [156]:
# run index autocoder selection on the acs data
acs_mrg['indexocc']=np.vectorize(indexautocode)(acs_mrg.occ1true,
                                                acs_mrg.occ1none,
                                                acs_mrg.occ2true,
                                                acs_mrg.occ3true,
                                                acs_mrg.occ3none)

The following cell shows the output for the first 20 records following the application of the `indexautocode()` function. Note that the output is a dictionary with keys "indexocc" and "match_code" referring the matched occupation code and match code, respectively.

In [157]:
# look at first few records
acs_mrg[['OCC','auto_uocc','ocw1','ocw2','occ1true','occ1none','occ2true','occ3true','occ3none','indexocc']].head(20)

,OCC,auto_uocc,ocw1,ocw2,occ1true,occ1none,occ2true,occ3true,occ3none,indexocc
0,4220,NaN,CONDO CLEANING,CLEANING CONDOS,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
1,9620,NaN,LABORER,WAREHOUSE,9620,9620,NaN,NaN,9620,"{'indexocc': '9620', 'match_code': 111}"
2,4220,NaN,MAINTENANCE,STOCKING/CLEANING,4220,NaN,NaN,NaN,NaN,"{'indexocc': '4220', 'match_code': 111}"
3,9130,9130,DRIVING A TRUCK,DRIVING THE TRUCK WHILE STRIPING THE ROAD,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
4,3422,NaN,BEHAVIORAL AIDE,TEACHING MENTAL PATIENTS BASIC SKILLS,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
5,7260,NaN,TIRE/BATTERY SPECIALIST,CHANGING TIRES & BATTERIES,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
6,4621,4621,PERSONAL TRAINING MANAGER,CLEARICAL WORK SHOWING MEMBERS ROUTINES AND EX...,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
7,4110,NaN,SERVING FOOD AND MAKING COCKTAILS,SERVING FOOD AND MAKING COCKTAILS,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
8,7340,NaN,MAINT,REPAIRS,NaN,NaN,NaN,NaN,NaN,"{'indexocc': None, 'match_code': None}"
9,0440,NaN,MANAGER,MANAGING,0335 0440,NaN,NaN,NaN,NaN,"{'indexocc': '0335 0440', 'match_code': 112}"


Next, I extract the components of the index autocode into their own variables and print the first 20 records for which a match can be made.

While the current index autocoder is able to produce unique matches, these results highlight an issue where industry restrictions are not resovling correctly. First, note the value for entry "9" referring to the "MANAGER" for `ocw1`. This case has industry code 8590, yet lists two possible occupation code assignments (`indexautoocc` = (0335, 0440)). The value ultimately assigned is `OCC` = 0440. However, the index has a value for "MANAGER" specifically referring to industry restriction "Gaming 8590" that is associated with occupation code 0335. Since the index autocoder here does not match on text, it should have still matched on `IND` = 8590 (I show this below following the table of results). A somewhat similar result happens for entry 22 for "REGISTRAR" that has an industry `IND` = 7860. Again, the index has this exclusion restriction but for some reason it is not being correctly controlled for by the `resolveind()` function earlier in this notebook. (This affects the entry for "MAIL CARRIER" as well.)

In [160]:
acs_mrg['indexautoocc'] = acs_mrg['indexocc'].apply(lambda x: x['indexocc'])
acs_mrg['indexautomcode'] = acs_mrg['indexocc'].apply(lambda x: x['match_code'])

acs_mrg[acs_mrg.indexautoocc.isnull()==False][['OCC','auto_uocc','IND','ocw1','ocw2','occ1true','occ1none','occ2true','occ3true','occ3none','indexautoocc','indexautomcode']].head(20)

,OCC,auto_uocc,IND,ocw1,ocw2,occ1true,occ1none,occ2true,occ3true,occ3none,indexautoocc,indexautomcode
1,9620,NaN,7580,LABORER,WAREHOUSE,9620,9620,NaN,NaN,9620,9620,111.0
2,4220,NaN,5381,MAINTENANCE,STOCKING/CLEANING,4220,NaN,NaN,NaN,NaN,4220,111.0
9,0440,NaN,8590,MANAGER,MANAGING,0335 0440,NaN,NaN,NaN,NaN,0335 0440,112.0
10,3255,NaN,8080,REGISTERED NURSE,NURSING,NaN,3255,NaN,NaN,NaN,3255,101.0
17,3500,NaN,8180,LICENSED PRACTICAL NURSE,PATIENT CARE,NaN,3500,3500,NaN,NaN,3500,101.0
20,6200,NaN,0770,SUPERVISOR,ROOFING - LABOR,6200,NaN,NaN,NaN,NaN,6200,111.0
22,5420,NaN,7860,REGISTRAR,RECORDS CLERK,5400 5420,NaN,NaN,NaN,NaN,5400 5420,112.0
23,7420,NaN,6680,TECHNICIAN,CUSTOMER SERVICE,7020,NaN,NaN,NaN,NaN,7020,111.0
25,4760,NaN,4971,STORE CLERK,STORE STOCK CLERK,9645,NaN,NaN,NaN,NaN,9645,111.0
26,9620,NaN,7580,LABORER,LABOR,9620,9620,NaN,NaN,NaN,9620,111.0


As mentioned, there are several cases where the current version of `resolveind()` is not isolating a single occupation code based on a battery of industry restrictions. Specifically, I mentioned entries associated with "REGISTRAR" and "MANAGER." Indeed, these are likely to be only a few of such cases. Below I present the occupation index entries for these cases. They show that the index could accurately assign these codes but the current state of the functions need to incorporate some bug fixes. Note item 11679, which applies to the above example.

In [167]:
occdf[occdf.title=='REGISTRAR']

,job_title,ind2017restr,cenocc,soc2018,hased,title,edrestr,indrestr
668,Registrar\ ns,7870,0230,11-9030,0,REGISTRAR,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
1124,Registrar\ ns,9370-9590,0440,11-9199,0,REGISTRAR,None,"[{'range': 'int({0}) in range(9370,9590+1)', '..."
1510,Registrar\ ns,7580,0630,13-1070,0,REGISTRAR,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
5205,Registrar\ ns,8570,2400,25-4010,0,REGISTRAR,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
11595,Registrar\ ns,\ Any not listed,5400,43-4171,0,REGISTRAR,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
11679,Registrar\ ns,7860,5420,43-4199,0,REGISTRAR,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."


In [163]:
occdf.iloc[11679]['indrestr']

[{'range': None,
  'cow': None,
  'exc': None,
  'code': '"{0}"=="7860"',
  'text': None}]

The following shows index entries for the job title "MANAGER." Note entry 765, which is associated with occupation code 0335 and would be the appropriate one to set using the index.

In [166]:
occdf[occdf.title=='MANAGER']

,job_title,ind2017restr,cenocc,soc2018,hased,title,edrestr,indrestr
83,Manager\ ns,6672,0020,11-1021,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
242,Manager\ ns,6695,0110,11-3021,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
290,Manager\ ns,6870-6890,0120,11-3031,0,MANAGER,None,"[{'range': 'int({0}) in range(6870,6890+1)', '..."
329,Manager\ ns,State employment office 7580,0136,11-3121,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
531,Manager\ ns,"GOV, OWN or PR 0170, 0180, 0280, 0290",0205,11-9013,0,MANAGER,None,"[{'range': None, 'cow': '{0} == [3, 4, 5]', 'e..."
659,Manager\ ns,"7860, 7870, 7880",0230,11-9030,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
660,Manager\ ns,8470,0230,11-9030,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
714,Manager\ ns,8680,0310,11-9051,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
723,Manager\ ns,9080,0325,11-9171,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
765,Manager\ ns,"Gaming 8590, 8660",0335,11-9070,0,MANAGER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."


In [168]:
occdf[occdf.title=='MAIL CARRIER']

,job_title,ind2017restr,cenocc,soc2018,hased,title,edrestr,indrestr
11746,Mail carrier,\ Any not listed,5510,43-5021,0,MAIL CARRIER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
11891,Mail carrier,6370,5550,43-5052,0,MAIL CARRIER,None,"[{'range': None, 'cow': None, 'exc': None, 'co..."
30101,Mail carrier,OWN 6170,9130,53-3030,0,MAIL CARRIER,None,"[{'range': None, 'cow': '{0} == [6, 7]', 'exc'..."


### Index Autocoder Results
In this section I provide summary statistics on the performance of the occupation index autocoder. First, I show the overall incidence (rate) of index autocodes based on whether a single occupation code or a list of occupation codes were selected. For cases with a unique match, I compare the accuracy of the occupation code to `OCC` that was originally assigned to the 2018 and 2019 data. For cases with multiple matches, I examine whether the "correct" (i.e, value of `OCC`) is within the list of chosen autocodes. 

Second, I examine the number of cases for which the index autocoder was able to make an occupation code assignment where the current logistic regresssion autocode was note. This provides a measure of cost savings over the current autocoder as the number of records that were originally sent to clerical coding but would not have to be based on the index.

Third, I compare accuracy rates between current autocode values to those from the index for cases where both autocoders are able to make predictions. This is a measure of data quality between the two regimes, where one has built into it a certain amount of probabilistic error while the other is deterministic.

#### Incidence of Index Autocode Assignment 
In this section I present the incidence of index autocode assignment in the combined panel of ACS 2018 and 2019 occupation write-ins for which write-in data are available.

First, let's get a distribution of match codes in the ACS file. I also create indicators for whether the match code corresponds to a unique occupation match code or whether a list of two or more occupation codes are returned. I also identify the source of the match.

In [179]:
# distribution of match types
match_codes = pd.DataFrame(acs_mrg.indexautomcode.value_counts()).reset_index()
match_codes.columns = ['Match Code', 'Frequency']
match_codes['Match Code'] = match_codes['Match Code'].apply(lambda x: int(x))
match_codes['match_type'] = np.where(match_codes['Match Code'].apply(lambda x: str(x)[-1]) == '2',
                                     'list',
                                     'unique')
match_codes['match_source'] = match_codes['Match Code'].apply(lambda x: 'ocw{}'.format(str(x)[0]))
match_codes

,Match Code,Frequency,match_type,match_source
0,101,711385,unique,ocw1
1,111,631908,unique,ocw1
2,112,209941,list,ocw1
3,211,51310,unique,ocw2
4,102,8917,list,ocw1
5,301,2447,unique,ocw3
6,311,1283,unique,ocw3
7,312,541,list,ocw3
8,302,29,list,ocw3


The above table shows the most matches occurred using just `ocw1`. Over 711,000 unique code assignments were based on write-ins with no education or industry restrictions (`Match Code` 101), while nearly 632,000 write-ins were matched to a unique occupation code with either an education or industry restriction or both (`Match Code` 111). An additional 21,000 cases returned a list of two or more occupation codes (`Match Code` 112), but as we saw in the previous section, some of these could be converted to unique matches after debugging the `resolveind()` function. Far fewer unique matches derive from `ocw2` and `ocw3`, but these fields are able to provide unique index autocode assignments.

Next, I present the overall match rate based on unique match codes. I find a unique occupation code can be matched to 30.8 percent of all write-ins. Compare that to the current logistic regression autocode that assigns a reliable autocode in about 50 percent of write-ins. This shows that a careful application of the occupation index can do about 60 percent of the work of the existing autocoder without the use of any probabilistic design features.

In [195]:
# overall match rate for unique matches across all sources
float(match_codes[(match_codes.match_type == 'unique')]['Frequency'].sum() / len( acs_mrg ))

0.308434671976891

Next, I calculate the share of write-ins that could be matched to two or more occupation codes from the index. Nearly 5 percent of records---which amounts to roughly 225,000 write-ins---can be assigned an autocode. As we demonstrated above, debugging elements of the index autocoder could convert some fraction of these to unique assignments.

In [194]:
# overall match rate for list matches across all sources
float(match_codes[(match_codes.match_type == 'list')]['Frequency'].sum() / len( acs_mrg ))

0.048399918476175016

Next, I present the distribution of unique index autocodes by source of the job title. About 96 percent of index autocodes come from `ocw1` alone, while 3.7 percent derive from `ocw2` and about a quarter of one percent can be derived from the composite string of `ocw1` and `ocw2`. This is interesting because some respondents enter their job titles in the field asking for task content. Further, the index is able to account for a combination of job titles and task content that has been reported by respondents in the past.

In [196]:
# distribution of unique matches by source
match_by_source = match_codes[match_codes.match_type=='unique'][['match_source','Frequency']].groupby('match_source').sum().reset_index()
match_by_source['Pct'] = match_by_source.Frequency.apply(lambda x: 100 * x/match_by_source.Frequency.sum())
match_by_source

,match_source,Frequency,Pct
0,ocw1,1343293,96.063885
1,ocw2,51310,3.669369
2,ocw3,3730,0.266746


#### Relative Match Rate of the Index Autocoder to the Current
One way to assess the relative benefits of the index autocoder to the existing logistic regression autocoder is to count the number of times it makes a unique prediction when the current production autocoder does not. This would highlight additional cases that would escape further clerical review that were ultimately sent to the NPC.

I use the ACS variable `auto_uocc` to determine whether the production autocoder assigned an occupation code for a given write-in. If the value of `auto_uocc` is missing, I assume that no code could be assigned, whereas a non-missing value indicates auto code assignment. The rate of autocode assignment can be calculated as the number of non-missing values of `auto_uocc` relative to the total write-ins on this file.

In [209]:
print('share of write-ins assigned a code with production autocoder:' ,
      len(acs_mrg[acs_mrg.auto_uocc.isna()==False]) / len(acs_mrg))

share of write-ins assigned a code with production autocoder: 0.288514492977393


The above shows that about 29 percent of write-ins were assigned an autocode based on the production autocoder. This number may be incorrect since materials provided by IOSB indicate that occupation autocode rates for the 2018 and 2019 survey years were between 40 and 43 percent. Nonetheless, I will proceed using this framework.

I showed in the previous section that the index autocoder was able to account for close to the same frequency of cases. But it is unclear how many cases the production autocoder was able to match that the index autocoder was not and vice versa. To that we now turn.


In [ ]:
# cases with a production autocode but no index autocode
prod_noind = len(acs_mrg[(acs_mrg.auto_uocc.isna()==False) & (acs_mrg.indexautoocc.isna())])

# cases with no production autocode but a index autocode
noprod_ind = len(acs_mrg[(acs_mrg.auto_uocc.isna()) & (acs_mrg.indexautoocc.isna()==False)])

# cases with both production autocode and index code
prod_ind = len(acs_mrg[(acs_mrg.auto_uocc.isna()==False) & (acs_mrg.indexautoocc.isna()==False)])

In [211]:
print('share with production code but no index code:' , prod_noind / len(acs_mrg))
print('share with no production code but index code:' , noprod_ind / len(acs_mrg))
print('share with production code and index code:' , prod_ind / len(acs_mrg))

share with production code but no index code: 0.13939272691018528
share with no production code but index code: 0.20771282438585825
share with production code and index code: 0.14912176606720776


The above results indicate that 14 percent of write-ins were assigned an occupation code by the production autocode but no the index autocode. About 21 percent of write-ins were assigned an occupation code by the index autocoder but not the production autocoder. Finally, 15 percent of write-ins were assigned a code by both the production autocoder and the index autocoder (we address whether the codes match in the next section).

These results demonstrate the potential benefits of mixing different methods to achieve higher match rates. For example, the production autocoder and the index autocoder only matched codes for about 30 percent of cases on their own. If the index autocoder were used with the production autocoder, it would be able to match close to 50 percent of cases based on these estimates alone.

#### Assessing Match Quality of the Index Autocoder
In this section, I breiefly examine the quality of index autocode assignments based on their comparisons to what was ultimately production autocoded or clerically coded. I compare the number incidence of cases where the index autocoder has a unique code assignment and that code assignment equals the production output value of `OCC`.

In [216]:
print('share of index autocodes that match OCC:' ,
      len(acs_mrg[(acs_mrg.indexautomcode.apply(lambda x: x in [101,111,211,301,311])==True) &
                  (acs_mrg.OCC==acs_mrg.indexautoocc)]) /
      len(acs_mrg[acs_mrg.indexautomcode.apply(lambda x: x in [101,111,211,301,311])==True]))

share of index autocodes that match OCC: 0.7999553754363231


The above shows that, for all cases where the index autocoder provided a unique occuption match, 80 percent of cases matched the ultimately assigned value of `OCC`. A future analysis could assess whether non-matching cases were assigned correctly either by the production autocoder or clerical coding. Further, some of the index autocoder matches could be spurious. Nonetheless, the index is a demonstrably effective tool for occupation autocoding.

### Summary
This notebook presented a method for utilizing the 2019 alphabetical index of occupation titles as an autocoder for the ACS occupation write-in fields. The index maintains a set of job titles along with, in some cases, specific education, class of worker, and industry code descriptions that direct which Census occupation code should be assigned to an occupation title matching that of the occupation index.

The index alone was shown to make unique occupation code predictions in about 31 percent of all ACS write-ins for the 2018 and 2019 1-year PUMS file. Of these unique write-ins, 80 percent matched what was ultimately coded on the output files. Current results from the internal ACS production autocoder show that between 40 and 43 percent of occupation write-ins are autocoded using the production autocoder. If my samples align with those, the index is able to account for nearly 75 percent of the workload of the production autocoder without estimating complicated models or imposing probabilistic error on coding assignments. While there is some overlap between the records that are coded using the production and index autocoders, I show that only 15 perent of records are matched by both. This suggests that utilizing multiple autocoding methods could achieve higher aggregate match rates that simultaneously increases the accuracy of autocode assignments and reduces the workload that must be sent to the NPC for clerical coding.

The index autocoder here uses a step-wise approach for matching index job titles based on the different ACS occupation write-in fields. Job titles are expected to match verbatim to those in the index and all explicit restrictions must be met (along with exceptions). I show that most job titles match on the `ocw1` field (96 percent of matches), while `ocw2` accounts for just under 4 percent. A small fraction (but numerious in actual records) can be matched on a composite of the fields.

The index autocoder develops specific functions to indentify restrictions and evaluate them against actual data. In its current state, the index autocoder performs well but suffers from some bugs where it is unable to distinguish a matching code amongst two or more potential matches despite having information to do so. This should be easy to correct. In addition, in its current state it does not leverage the full set of restrictions or exceptions listed in the index, namely explicit references to "titles within industry codes" or other text. A future version of the index autocoder should incorporate these elements.

This work demonstrates that the occupation index can serve as a useful tool in developing occupation autocodes moving forward. In particular, I envision it as a first stage in an autocoder ensemble where job title matches are made from the index as a first pass and set aside, while the remaining non-matches are sent through a more formal ML prediction model. To the extent that the index is accurate and correctly parsed into code, its matches should be of the highest quality and effectively error-free. In addition, the functions used here were able to make autocode assignments on the full set of 4.5 million ACS write-in records in under 10 minutes. Finding the solution to an optimal ML model would take considerably more time on a small subset of these records depending on the method used. 

Finally, the ability of the index to serve as an autocoder depends on how exhaustive it is, whether titles are mutually exclusive conditional on industry, education, and COW restrictions, and how easy it is to parse this information from it. The index is, for the most part, easy to parse, however some complications are present. I exploited pattern recognition tools to identify education references (which are not standard---but close!), as well as leveraged the fact that multiple industry restrictions for a given job title were *typically* separated by a comma. These patterns are not always met and may impose some restrictions that are not as intended. The index, when used in this capacity, could benefit from some additional design features that take these issues into account.